# Updates/Installation

In [1]:
'''


#Need to update? Here ya go!!


#!py -m pip uninstall astropy
#!py -m pip install git+https://github.com/astropy/astropy
#!pip install emcee
!pip install corner
    



!py -m pip install git+https://github.com/radio-astro-tools/spectral-cube.git
!py -m pip install reproject
!py -m pip install git+https://github.com/radio-astro-tools/spectral-cube.git 
!py -m pip install pyspeckit
!py -m pip install regions
!py -m pip install astrodendro
!py -m pip  install wcsaxes 
!py -m pip  install ipympl
!py -m pip install dask
!py -m pip install radio_beam
!py -m pip install casa_formats_io
#try:
#    !pip install casa_formats_io --no-binary :all:
#except:
#    !pip install casa_formats_io --no-cache --no-binary :all:

!py -m pip  install spectral_cube 
!py -m pip  install typing 
!py -m pip install mypy
!py -m pip  install typing_extensions 



'''


/bin/bash: line 1: pip: command not found


# Imports

In [2]:
######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################

#Every data reduction and analysis file will use these imports and functions.
#So i run this file at the beggining of every other file to import stuff.

######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################

#These will show you what version of Python you are working with. Important because astropy works best with certain versions like 3.8.5

import sys, traceback

print(sys.executable)
print(sys.version)
print(sys.version_info)



import math
import os
import copy

# The most important package for astronomy

import astropy
from astropy.coordinates import SkyCoord
print('astropy',astropy.__version__ )
import astropy.io.fits as fits              
from astropy.wcs import WCS # World coordinate system
from astropy import units as u  
from astropy.table import Table
from astropy.convolution import Gaussian1DKernel
from astropy.utils import NumpyRNGContext
from astropy.coordinates import Angle


# Spectral cubes are amazing at taking fits images and turning them into workable position-position-velocity cubes 
# (The cubes are 3D, but the moment maps/continuum images will be 2D and SC will also work for them)

from spectral_cube import SpectralCube    
from spectral_cube import LazyMask
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
print('spectral_cube file path',spectral_cube.__file__)

# Need this for projection of the cubes

from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)

# Useful for doing analysis

import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import matplotlib.colors as colors
from matplotlib import pyplot as plt
import scipy
import numpy as np                          
from matplotlib.patches import Ellipse
print(matplotlib.__version__,"Matplotlib")
print(matplotlib.__file__)
print(np.__version__,"Numpy")
from scipy.optimize import curve_fit
from scipy.optimize import leastsq
from scipy.spatial.transform import Rotation as R # For doing 3d rotations

# astrodendro is a key package, it allows a quick way to identify structures

import astrodendro 
from astrodendro.analysis import PPVStatistic # Takes statistics of PPV structures

print("astrodendro_file:", astrodendro.__file__)

# The radio_beam library allows you to check/change teh interferometric properties of the beam.

import radio_beam

# Garbage collection

import gc

# Suppress warnings we don't care about:

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

    
#     

from tqdm import tqdm

#
# Create the save directories
#

if(os.path.exists("./Result Files")):
    print("Results will be saved to Directory ./Result Files")
else:
    %mkdir "./Result Files"
    print("Created directory ./Result Files where results will be saved")
    
if(os.path.exists("./Spectral Cubes")):
    print("Cubes will be saved and loaded with Directory ./Spectral Cubes")
else:
    %mkdir "./Spectral Cubes"
    print("Created directory ./Spectral Cubes where cubes will be saved/loaded")
    
if(os.path.exists("./Spectral Cubes/Cube Information")):
    pass
else:
    %mkdir "./Spectral Cubes/Cube Information"
    print("Created directory ./Spectral Cubes/Cube Information where cube information will be saved")
    
if(os.path.exists("./Plots")):
    print("Plots will be saved to Directory ./Plots")
else:
    %mkdir "./Plots"
    print("Created directory ./Plots")
    
if(os.path.exists("./Dendrograms")):
    print("Dendrograms will be saved to Directory ./Dendrograms")
else:
    %mkdir "./Dendrograms"
    print("Created directory ./Dendrograms")
# If you need to interact with teh plots, use widget, otherwise this runs better

%matplotlib inline
#%matplotlib widget 
sys.setrecursionlimit(9999999)  # for very large functions


/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
spectral_cube 0.6.1.dev22+g003ef16
spectral_cube file path /home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
reproject 0.8
1.23.1 Numpy
astrodendro_file: /home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
Results will be saved to Directory ./Result Files
Cubes will be saved and loaded with Directory ./Spectral Cubes
Created directory ./Plots


In [ ]:
# Constants (unused)

Num_per_kg= 6.0221409*10**23/(2.8*10**-3) /u.kg # A number that Krieger used for number/kg of H2

a_850 = 6.7*10**19*u.erg/u.s/u.Hz/u.M_sun #6.7+-1.7, from Bolatto 2013a, for converting continuum flux to mass 


# Functions

Make general functions for each process so I can call them simply.
The only things I need to change are the input files and their properties

# Pointing Information

In [ ]:
# Pointing_Information


# Pointing_Information an important variable that will need to be defined for each source
# It is a dictionary containing all the metadata and pointing information. This is important because the PPVstatistic needs it to be in a specific format
#
# eg Pointing_Information:  
'''

Pointing_Information = {}

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "HCN_J1-0.cube.fits" #the name of the initial SC file.
Pointing_Information["File_Descriptor"] = "NGC_253_HCN_J1-0_"
Pointing_Information["target"] = "NGC253"#or the cmz
Pointing_Information["center"] = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs') #the center of NGC253
#I use center = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') for the cmz center
desired_beam_size = 4.3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 3.5*u.Mpc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=33*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=78*u.deg
Pointing_Information["target_velocity"]=250*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"]= 3.3*u.km/u.s
ovs = 3 #how much do you desire to oversample the beam by
desired_fov = [360*u.pc,70*u.pc]
Pointing_Information["Corresponding_Continuum"] = 'Continuum_Reproject.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2)
#                                                 "gc_850micron_dust.fits" # this is the CMZoom dust continuum from SCUBA Galactic Centre Survey (850um)

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header

try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=header["RESTFREQ"]#            
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=header["RESTFRQ"]#    

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 
    
beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec)


Pointing_Information["original_BMAJ"]=beam_major
Pointing_Information["original_BMIN"]=beam_minor
Pointing_Information["original_BMAJ_pc"]=beam_major.to(u.rad)*Pointing_Information['distance']
Pointing_Information["original_BMIN_pc"]=beam_minor.to(u.rad)*Pointing_Information['distance']
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size


#this accounts for elliptical beams:            
Pointing_Information['original_pixel_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_pixel_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_spatial_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']
Pointing_Information['original_spatial_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']#convert to pc using the distance

average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

Pointing_Information['original_beam_oversampling_MAJ'] = beam_major/average_pixel
Pointing_Information['original_beam_oversampling_MIN'] = beam_minor/average_pixel
Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target'],Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_fov,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######


#Cube_Information. This needs to be updated every time a reduction occurs on the cube. By the end of file 1 which does all the reprojection, this will not change.
#The pointing information will not change, so I'll start with a copy of that:

Cube_Information = copy.deepcopy(Pointing_Information)

#For example, after doing the reprojection, I need to load: 


Current_Cube_Name = Pointing_Information["File_Descriptor"] + str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'++str(i)+'.fits'
Cube_Information["File_Name"]=Current_Cube_Name

sc = SpectralCube.read(Current_Cube_Name)
header = sc.header

pc_per_pixelx = abs(header["CDELT1"]*Cube_Information["CUNIT"].to(u.rad))*Pointing_Information['distance']/u.pix #convert degrees to size using the distance
pc_per_pixely = abs(header["CDELT2"]*Cube_Information["CUNIT"].to(u.rad))*Pointing_Information['distance']/u.pix #convert degrees to size using the distance

Cube_Information["pixel_scale_x"]=pc_per_pixelx
Cube_Information["pixel_scale_y"]=pc_per_pixely

Cube_Information['data_unit'] =sc[0][0][0].unit# or just use sc.header['BUNIT']

Cube_Information['arc_per_pix_y'] =  abs(header["CDELT1"]*CUNIT).to(u.arcsec)/u.pix 
Cube_Information['arc_per_pix_x'] =  abs(header["CDELT2"]*CUNIT).to(u.arcsec)/u.pix

beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec) #if these two are different, something went wrong in the reprojection
Cube_Information['beam_major'] =  beam_major
Cube_Information['beam_minor'] =  beam_minor

beam_area_ratio = beam_minor*beam_major/Cube_Information['arc_per_pix_y']/Cube_Information['arc_per_pix_x']*1.13309#beam_area_ratio = Cube_Information['beam_minor']*Cube_Information['beam_major']/Cube_Information['arc_per_pix_y']/Cube_Information['arc_per_pix_x']#This is for FWHM, use *(2*np.sqrt(2*np.log(2)))**2#For gaussian beam
Cube_Information['beam_area_ratio']=beam_area_ratio #This is important for finding the minimum number of pixels a structure can have, or for calculating column densities

#this accounts for elliptical beams:            
Cube_Information['spatial_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']
Cube_Information['spatial_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']
                                     
                                     



Cube_Information["velocity_scale"] = abs(header["CDELT3"])*u.km/u.s#the cube should be in u.km/u.s instead of frequency on the z axis

average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

Cube_Information['beam_oversampling'] = beam_minor/average_pixel
Cube_Information['desired_beam_oversampling'] = Pointing_Information['desired_beam_oversampling']

Cube_Information["wcsu"]=sc.wcs






'''

#make a function to do this easily

def Update_Cube_Information(Pointing_Information,Current_Cube_Name):

    Cube_Information = copy.deepcopy(Pointing_Information)

    Cube_Information["File_Name"]=Current_Cube_Name
    try:
        del sc
        print("Deleted sc")
    except:
        pass
    
    try: 
        try:
            sc = SpectralCube.read("Spectral Cubes/"+Current_Cube_Name) #For a 3d cube
        except:
            sc = SpectralCube.read("Spectral Cubes/"+Current_Cube_Name,format='fits') #For a fits cube that is not labelled as such and is 3d
    except: 
        sc = spectral_cube.Projection.from_hdu(fits.open("Spectral Cubes/"+Current_Cube_Name)) # For a 2d cube
    header = sc.header

    if (header['CUNIT1'].find("deg")!=-1):
        CUNIT = 1*u.degree
        Pointing_Information["CUNIT"]=CUNIT
    else:
        print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
        del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 
    
    #pc_per_pixelx = abs(header["CDELT1"]*Cube_Information["CUNIT"].to(u.rad))*Pointing_Information['distance']/u.pix #convert degrees to size using the distance
    #pc_per_pixely = abs(header["CDELT2"]*Cube_Information["CUNIT"].to(u.rad))*Pointing_Information['distance']/u.pix #convert degrees to size using the distance

    #Cube_Information["pixel_scale_x"]=pc_per_pixelx
    #Cube_Information["pixel_scale_y"]=pc_per_pixely
    
    try:
        Cube_Information['data_unit'] =sc[0][0][0].unit# or just use sc.header['BUNIT']
    except:
        Cube_Information['data_unit'] =sc[0][0].unit# or just use sc.header['BUNIT']

    Cube_Information['arc_per_pix_y'] =  abs(header["CDELT1"]*CUNIT).to(u.arcsec)/u.pix 
    Cube_Information['arc_per_pix_x'] =  abs(header["CDELT2"]*CUNIT).to(u.arcsec)/u.pix

    try:
        
        # Get the beam size. If there is none, use the pixel size
        beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
        beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec) #if these two are different, something went wrong in the reprojection 
        Cube_Information['Beam_Position_angle'] = header['bpa']*u.deg

        
    except:
        
        cdelt_x = u.Quantity(str(np.abs(header['cdelt1']))+header['cunit1'])
        cdelt_y = u.Quantity(str(np.abs(header['cdelt2']))+header['cunit2'])
        if(cdelt_x>cdelt_y):
            beam_major=cdelt_x
            beam_minor=cdelt_y
        elif(cdelt_x<cdelt_y):
            beam_major=cdelt_y
            beam_minor=cdelt_x
        elif(cdelt_x==cdelt_y):
            beam_major=cdelt_x
            beam_minor=cdelt_x
        
        Cube_Information['Beam_Position_angle'] = 0*u.deg

    
    Cube_Information['beam_major'] =  beam_major
    Cube_Information['beam_minor'] =  beam_minor

    beam_area_ratio = beam_minor*beam_major/Cube_Information['arc_per_pix_y']/Cube_Information['arc_per_pix_x']*1.13309#beam_area_ratio = Cube_Information['beam_minor']*Cube_Information['beam_major']/Cube_Information['arc_per_pix_y']/Cube_Information['arc_per_pix_x']#This is for FWHM, use *(2*np.sqrt(2*np.log(2)))**2#For gaussian beam
    Cube_Information['beam_area_ratio']=beam_area_ratio.value*u.pix**2/u.beam #This is important for finding the minimum number of pixels a structure can have, or for calculating column densities
    
    
    
    Cube_Information['spatial_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance'] / u.rad
    Cube_Information['spatial_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance'] / u.rad #pc/pixel
    
    #this is a specific value used by the dendrogram, and they want it in (angle)
    Cube_Information['spatial_scale'] = np.sqrt(Cube_Information['arc_per_pix_x']*Cube_Information['arc_per_pix_y'])*u.pix

    try:
        Cube_Information["velocity_scale"] = abs(header["CDELT3"])*u.km/u.s#the cube should be in u.km/u.s instead of frequency on the z axis
    except:
        pass

    average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

    Cube_Information['beam_oversampling'] = beam_minor/average_pixel
    Cube_Information['desired_beam_oversampling'] = Pointing_Information['desired_beam_oversampling']

    Cube_Information["wcsu"]=sc.wcs
    
    #Cube_Information['FOV']= Crop_Around_Center(sc,Cube_Information['target_image_rotation'],Cube_Information['center'],Cube_Information['desired_FOV'],Cube_Information['distance'])[1] #returns [the cropped sc, the current fov, the desired fov]
    #Cube_Information['desired_FOV']=Pointing_Information['desired_FOV']
    
    return Cube_Information

# Reprojection function

To align the files across and get the same beam size and FOVs along different observations

In [ ]:

#Spatial reprojection 

# File = the file name of the SC you want to reproject
# Prime_Beam = the pc size of the beam. When comparing a line in two different sources, i match the beam sizes across the sources

# Pointing_Information= see earlier
# Cube_Information= see earlier
# i_step = the amount of velocity channels for each step in the reprojection. lower this if your computer lags a lot.

def Reproject_To_Region(Pointing_Information,Cube_Information,i_step=30,Cube_Name_Save='',Force_Origin=[False,[0,0]*u.deg,[0,0]*u.deg],partway = False,crop_nans=True,change_origins=True,force_pixels=False,fy=200,fx=200,GLON_BASE=False):
    
    File = Cube_Information["File_Name"]
    Prime_Beam = Pointing_Information["desired_beam_size"]
    Gal = Pointing_Information["target"]
    ovs = Pointing_Information["desired_beam_oversampling"]
    FOV = Pointing_Information["desired_FOV"]
    distance=Pointing_Information["distance"]
    target_velocity= Pointing_Information["target_velocity"]
    try:
        center = Pointing_Information["crop_center"] # If there is a specified cropping center, use it.
    except:
        center = Pointing_Information["center"]
    rotation_angleP = Pointing_Information["target_image_rotation"]
    Line_Name = Pointing_Information["File_Descriptor"]

    run_completed=False
    # These are related to error correction the original CMZ cubes because they have an error with their longitude values going from 360->0 through the center
    
    Force_Origins=Force_Origin[0]
    Force_Value_x=Force_Origin[1]
    Force_Value_y=Force_Origin[2]
    
    #Load observation cube
    
    try:
        scB = SpectralCube.read("Spectral Cubes/"+File) 
    except:
        scB = SpectralCube.read("Spectral Cubes/"+File,format='fits') 
    scB.allow_huge_operations=True
    
    scB = scB.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s in case they are in Hz
    
    #Determine the resolution based on the oversample factor
    
    beam_deg =  ((Prime_Beam/(distance))*u.rad).to(u.deg)#deg corresponding to the desired beam size
    
    # Check the coordinate systems
    
    vel,RA,Dec = scB.world[:,0,0]
    
    # Need to break it up into 30-wide vel slices to do the reprojection (ram-draw too high otherwise)  
    
    for i in range(int(len(scB)/i_step) +1):
        
        try:
            print('begin step:',i,"of",int(len(scB)/i_step) +1)
            
            Cube_Name_Save = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOV[0].value)+"x"+str(FOV[1].value)+'pc_'+'reprojected.fits'
            
            if os.path.exists(("Spectral Cubes/"+str(str(i)+"_"+Cube_Name_Save))):
                if partway:
                    continue #If there are steps that have already been completed, skip them
            
            n = scB[i*i_step:i*i_step+i_step]
            n.write("h_interim.fits",overwrite=True)
            #sc = copy.deepcopy(scB[i*i_step:i*i_step+i_step])
            sc = SpectralCube.read("h_interim.fits") #need to do this because python has glitches with pointers and copying
            
            vel_P,xx,xxxx = sc.world[:,0,0]
            if(min(vel_P.to(u.km/u.s).value) > (target_velocity + 251*u.km/u.s).value or max(vel_P.to(u.km/u.s).value) < (target_velocity - 251*u.km/u.s).value):
                if run_completed:
                    break #If the data is out of the velocity bounds, stop the loop
                else:
                    del sc
                    del vel_P
                    continue
            sc.write("test1.fits",overwrite=True)

            sc = sc.spectral_slab(target_velocity- 251*u.km/u.s, target_velocity+ 251*u.km/u.s)  # Crop out velocities we don't care about  
            
            sc.allow_huge_operations=True
            sc.write("test2.fits",overwrite=True)
            
            '''
            #
            #
            # Rotate cube to flat
            #
            #
            '''
            # First determine the end bounds of the cube
            long_extrema = sc.longitude_extrema
            lat_extrema = sc.latitude_extrema
            
            rotation_angle = rotation_angleP # The original rotation
            if(rotation_angle!=0*u.deg):
                
                print('starting cube rotation')
                sc_R =  Rotate_Cube(sc, rotation_angle) # Rotates the cube to flatten it. This might take a while
                del sc
                sc = SpectralCube.read(sc_R.hdu)
                del sc_R
                rotation_angle = 0*u.deg # New rotation angle (The cube is flat now)
                Pointing_Information["target_image_rotation"] = rotation_angle # This is not really pointing information but I need it here
                print('finished cube rotation')


            
            print('start beam convolution')

            try:
                #Make a circular beam to convolve the image to.
    
                beam = radio_beam.Beam(major=beam_deg, minor=beam_deg, pa=0*u.deg)
                sc = sc.convolve_to(beam)
            
            except:
                print("The initial image has no beam because it's not interferometric, so one will be created using the pixel size as the initial beam size")
                
                cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
                cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
                if(cdelt_x>cdelt_y):
                    majorBase=cdelt_x
                    minorBase=cdelt_y
                elif(cdelt_x<cdelt_y):
                    majorBase=cdelt_y
                    minorBase=cdelt_x
                elif(cdelt_x==cdelt_y):
                    majorBase=cdelt_x
                    minorBase=cdelt_x
                    
                BaseBeam = radio_beam.Beam(major=majorBase, minor=minorBase, pa=0*u.deg)

                sc = sc.with_beam(BaseBeam)
                beam = radio_beam.Beam(major=beam_deg, minor=beam_deg, pa=0*u.deg)

                sc.allow_huge_operations=True
                #Requires me to edit convolve.py and set allow_huge =True
                #If this fails, go edit that file in your repository.
                sc = sc.convolve_to(beam)
                
            print('convolve end\n')
            
            
            
            #Mask the pixels outside the fov
            
            #returns: cropped_sc,orig_fov,FOV
            print('fov crop start \n')
            
            #sc.write("test3.fits",overwrite=True)
            

            cropped_sc = Crop_Around_Center(sc,rotation_angle,center,FOV,distance,crop_nans=crop_nans)
            del sc
            sc = cropped_sc
            sc.write("test4.fits",overwrite=True)
            del cropped_sc
            print('check max SC value after crop:',np.nanmax(sc),"SC shape:", np.shape(sc))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
            print('fov cropped\n')
            
            
            #
            #prepare a header for the reprojection
            #
            
            reheader = copy.deepcopy(sc.hdu.header)
            

            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 

            #find out which direction the cube is read, left to right or right/to left. (in terms of RA/DEC). Then, do the same for up and down
            
            if sc.header['cdelt1']>0:
                pix_x    = (beam_deg/ovs).to(u.degree).value
                #origin_x = sc.longitude_extrema[0].to(u.degree).value  #, this is the current value, but it doesnt work with the rotation so I use the original values
                origin_x = long_extrema[0].value
                
                if(Force_Origins):
                    origin_x = Force_Value_x[0]#358.6

            else:
                pix_x    = -1.*(beam_deg/ovs).to(u.degree).value
                #origin_x = (sc.longitude_extrema[1]).to(u.degree).value #, this is the current value, but it doesnt work with the rotation so I use the original values:
                origin_x = long_extrema[1].value

                if(Force_Origins):
                    origin_x = Force_Value_x[1]#.9

            if sc.header['cdelt2']>0:
                pix_y    = (beam_deg/ovs).to(u.degree).value
                #origin_y = sc.latitude_extrema[0].to(u.degree).value  #, this is the current value, but it doesnt work with the rotation so I use the original values
                origin_y = lat_extrema[0].value
                if(Force_Origins):
                    origin_y = Force_Value_y[0]#-.6
                    
            else:
                pix_y    = -1.*(beam_deg/ovs).to(u.degree).value
                #origin_y = sc.latitude_extrema[1].to(u.degree).value  #, this is the current value, but it doesnt work with the rotation so I use the original values
                origin_y = lat_extrema[1].value

                if(Force_Origins):
                    origin_y = Force_Value_y[1]#.6
                    
        
            #npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            #npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            #npix_x   =int(np.ceil(np.diff([origin_x,origin_x_max])/np.abs(pix_x)))
            #npix_y   =int(np.ceil(np.diff([origin_y,origin_y_max])/np.abs(pix_y)))
            #manually make the cubes overproject because it cuts off data otherwise, because it cannot properly convert the longitude/latitude extrema to ra/dec information:
            try:
                npix_x   =int(np.ceil(np.diff([long_extrema.value])/np.abs(pix_x)))
                npix_y   =int(np.ceil(np.diff([lat_extrema.value])/np.abs(pix_y)))
            except:
                print("Longitude and lattitude values not found.")
                npix_x = 200
                npix_y = 200 #default size
                
            crpix1 = 0
            crpix2 = 0
            
            if(Force_Origins):
                
                npix_x   =abs(int(np.diff([Force_Value_x[0]-360,Force_Value_x[1]])/np.abs(pix_x)) )
                npix_y   =int(np.diff(Force_Value_y)/np.abs(pix_y)) 
                print("Force values of:", Force_Value_x,Force_Value_y,"Giving pixel range of ",npix_x,npix_y)
            
            if(rotation_angleP != 0*u.deg):
                #if there is a rotation, i need additional fov to account for it
                npix_x +=400 #increase the axis size
                npix_y +=200
                crpix1 +=400 # the anchor pixel for the minimum longitude
                crpix2 +=200 # for latitude
            if(force_pixels):
                npix_y=fy
                npix_x=fx
            #Correct the header to the expected pixels for the new res

            reheader['cdelt1'] = pix_x
            reheader['cdelt2'] = pix_y

            reheader['naxis1'] = npix_x
            reheader['naxis2'] = npix_y
            
            
            if(change_origins):
                
                reheader['crval1'] = origin_x
                reheader['crval2'] = origin_y

                reheader['crpix1'] = crpix1
                reheader['crpix2'] = crpix2
            
            if(str(sc.wcs).find("GLON-SIN")!=-1):
                reheader['CTYPE1'] = "GLON-SIN"
                reheader['CTYPE2'] = "GLAT-SIN"
            elif(str(sc.wcs).find("GLON-TAN")!=-1):
                reheader['CTYPE1'] = "GLON-TAN"
                reheader['CTYPE2'] = "GLAT-TAN"
            elif(str(sc.wcs).find("GLON-CAR")!=-1):
                reheader['CTYPE1'] = "GLON-CAR"
                reheader['CTYPE2'] = "GLAT-CAR"
            elif(str(sc.wcs).find("GLON")!=-1 and GLON_BASE==False):
                reheader['CTYPE1'] = "GLON-SIN" #SIN is the default for GLON
                reheader['CTYPE2'] = "GLAT-SIN"                
            elif(str(sc.wcs).find("GLON")!=-1 and GLON_BASE):
                reheader['CTYPE1'] = "GLON" #SIN is the default for GLON
                reheader['CTYPE2'] = "GLAT"
                
            # remove these things that confuse the reprojection since we won't be using them
            try:
                del reheader['lonpole']
                del reheader['latpole']
                del reheader['wcsaxes']# Dont need these anymore
                if(str(sc.wcs).find("GLON")!=-1):

                    del reheader['LBOUND1']
                    del reheader['LBOUND2']
                    del reheader['LBOUND3']
                    del reheader.cards['LBOUND1']
                    del reheader.cards['LBOUND2']
                    del reheader.cards['LBOUND3']

                    reheader['LBOUND1']=0
                    reheader['LBOUND2']=0
                    reheader['LBOUND3']=0
            except Exception as e:
                print("Delete LBOUND if there is one in the header, if not this will pass the error:")
                print(e)
                print("-"*60)
                traceback.print_exc(file=sys.stdout)
                print()

            # regrid cube to target pixel size

            print('start reprojection\n')
            print('check max SC value:',np.nanmax(sc),"SC shape:", np.shape(sc))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
            sc.write("test5.fits",overwrite=True)
            sc.allow_huge_operations=True
            sc2 = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True) # Had to change reproject.py so it deletes output.np before making a new one
            sc2.write("test6.fits",overwrite=True)

            del sc # save space

            # make a new cube with the reprojcted data (remove all the logs from the old cube)
            
            new = SpectralCube(data=sc2.hdu.data,wcs =WCS(sc2.header),header=sc2.header,mask=sc2.mask)
            new.allow_huge_operations=True
            new = new*sc2[0][0][0].unit
            
            #do this because scs dont like being modified
            del sc2
            sc2 = new
            del new
            
            print('\nend reprojection\n')
            print('check max SC value:',np.nanmax(sc2),"SC shape:", np.shape(sc2))#These should ALSO be a non zero float and the shape of the cube (30,~1000,~1000)

            sc = sc2
            del sc2
            sc.allow_huge_operations=True
            
            if(crop_nans):
                #Mask the pixels outside the fov again after the reprojection to get rid of nan created pixels

                print('fov crop start 2 \n')

                cropped_sc = Crop_Around_Center(sc,rotation_angle,center,FOV,distance)
                del sc
                sc = cropped_sc
                del cropped_sc
                print('fov cropped 2\n')
                # Write the intermediary cubes that will be spliced together
            
            sc.write("Spectral Cubes/"+str(str(i)+"_"+Cube_Name_Save),overwrite=True)
            
            run_completed=True
            del sc
        except Exception as e:
            print(e)
            print("Failed (unless this says attempt to get argmin of empty sequence)")
            print("-"*60)
            traceback.print_exc(file=sys.stdout)

            


In [ ]:

#############
# Crop_Around_Center
#############


#This function crops out things outside the rectangle where the actual disk lies.
#The disk may be rotated, so this will be a rotated rectangle.
#Without a circular beam, this will be off, but I only use it after I circularize the beam.


#returns: cropped_sc,orig_fov,FOV

def Crop_Around_Center(sc_for_cropping,rotation_angle,center,desired_fov,distance,crop_nans=True):

    
    
    r,c,FOV,d = rotation_angle,center,desired_fov,distance
    
    
    cdelt_x = u.Quantity(str(np.abs(sc_for_cropping.header['cdelt1']))+sc_for_cropping.header['cunit1'])
    cdelt_y = u.Quantity(str(np.abs(sc_for_cropping.header['cdelt2']))+sc_for_cropping.header['cunit2'])
    center_ra_pix,center_dec_pix = [int(sc_for_cropping.wcs[:][:][0].world_to_pixel(c)[0]),int(sc_for_cropping.wcs[:][:][0].world_to_pixel(c)[1])]
    PixFov = [int((FOV[0].to(u.pc)/(cdelt_x.to(u.rad)*d.to(u.pc))).value/2),int((FOV[1].to(u.pc)/(cdelt_x.to(u.rad)*d.to(u.pc))).value/2)] #they'll be in pixels, but I only need the int

    print("Center:",c,"Pixel center:",center_ra_pix,center_dec_pix,"Pixel FOV:",PixFov)
    
    pixels = np.zeros(np.shape(sc_for_cropping))           
    
    print("cropping cube. rotation:",r,"center:",center,"crop to:",desired_fov)
    
    if(r!=0*u.deg):
        #to save time
        r_rad=r.to(u.rad).value
        #Find the pixels that are outside the rectangular FOV
        for lmj in range(np.shape(sc_for_cropping)[1]):
            for lmk in range(np.shape(sc_for_cropping)[2]):

                #The hypotenuse
                hypo = np.sqrt(((lmj-center_dec_pix)**2) + (lmk-center_ra_pix)**2)

                if (lmj-center_dec_pix!=0):
                    ang = np.arctan(abs(lmk-center_ra_pix)/abs(lmj-center_dec_pix))#*u.rad#Find angle to the center
                else:
                    ang = np.pi/2#*u.rad
                if(lmk>center_ra_pix and lmj>center_dec_pix):
                    ang*=-1
                elif(lmk<center_ra_pix and lmj<center_dec_pix):
                    ang*=-1
                elif(lmk>center_ra_pix and lmj<center_dec_pix):
                    if ang > (np.pi/2-r_rad):

                        ang= -r_rad+(np.pi-r_rad)-ang#coming from the opposite end of the ra axis now, but projecting still to 33 deg from north.
                    else:
                        pass


                up_pixels = abs(hypo*np.cos(abs(r_rad+ang)))
                side_pixels = abs(hypo*np.sin(abs(r_rad)+ang))

                #Check if the pixels are inside the FOV
                if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):
                    for lmi in range(np.shape(sc_for_cropping)[0]):
                        pixels[lmi][lmj][lmk] = 1  # keep this pixel
                
    else:
        
        # If the image is not rotated, the FOV is just a rectangle
        
        # Find all pixels in the fov
        
        for lmj in range(np.shape(sc_for_cropping)[1]):
            for lmk in range(np.shape(sc_for_cropping)[2]):

                up_pixels = abs(lmj-center_dec_pix)#Should not be over the fov in the upwards direction (relative to 0 degrees)
                side_pixels = abs(lmk-center_ra_pix)#Should not be over the fov in the side-side direction (relative to 0 degrees)

                if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):
                    for lmi in range(np.shape(sc_for_cropping)[0]):
                        pixels[lmi][lmj][lmk] = 1  # keep this pixel
                        
    # Mask teh pixels outside the fov

    bp = np.where(pixels!=1)
    scCopy = sc_for_cropping.hdu
    scCopy.data[bp]=np.nan
    scP = SpectralCube.read(scCopy)
    del scCopy
    del bp
    #Get right size by removing the nan pixels
 
    scP.allow_huge_operations=True
    datn = scP.hdu.data
    dat_sum = np.nansum(datn,axis=0)
    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(dat_sum[:,:])[0]):

        # Go through a slice of the cube and find the first pixels with rael data
        
        # After this is done these will be non zero so break the loop
        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            
            break
            
        for lmj in range(np.shape(dat_sum[:,:])[1]):

            if(sx==0):            
                if(np.nanmean(dat_sum[lmi,:])>0 or np.nanmean(dat_sum[lmi,:])<0):
                    sx=lmi

            if(sy==0):
                if(np.nanmean(dat_sum[:,lmj])>0 or np.nanmean(dat_sum[:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(dat_sum[np.shape(dat_sum[:,:])[0]-lmi-1,:])>0 or np.nanmean(dat_sum[np.shape(dat_sum[:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(dat_sum[:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(dat_sum[:,np.shape(dat_sum[:,:])[1]-lmj-1])>0 or np.nanmean(dat_sum[:,np.shape(dat_sum[:,:])[1]-lmj-1])<0):
                    ey=np.shape(dat_sum[:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break
    if crop_nans:
        sc1 = scP[:,sx:ex,sy:ey] #get rid of the nan pixels
    else:
        sc1 = scP
    scP_Hdu=sc1.hdu
    
    
    # Also check if there are zeroes in place of nan values, which is done on some cubes
    
    zeros=((scP_Hdu.data[:,:,:]==0))
    bp = np.where(zeros)
    scP_Hdu.data[bp]=np.nan
    scCopy = SpectralCube.read(scP_Hdu)
    del scP_Hdu
    
    # Make the final cropped cube

    cropped_sc = SpectralCube.read(scCopy.hdu)
    
    del scCopy
    
    data=sc_for_cropping.hdu.data
    
    print("Cropped to ",FOV, "pixels",sx,ex,sy,ey)
    
    return cropped_sc

In [ ]:

# Splice the partwise cubes back together:
    
def Splice_vels(Cube_Name_Load,istart=0,iend=2000,Save='',crop=list([[0,99999999999],[0,99999999999]])):
    missed=False
    hit=False
    for i in range(0,iend):
        gc.collect()
        try:
            Cube_Name_Load_p = str(i)+"_"+Cube_Name_Load
            Cube_Name_Save = Cube_Name_Load

            sc=SpectralCube.read(("Spectral Cubes/"+Cube_Name_Load_p))[:,crop[1][0]:crop[1][1],crop[0][0]:crop[0][1]]
            hit=True
            if istart ==0 and missed:
                istart=i
            print("Loaded",Cube_Name_Load_p)
            print
            
            # Define a header that we will form into the new header for the spliced cube
            if i == istart:
                reheader = sc.header
                rewcs = sc.wcs

            if i == istart:
                scW=SpectralCube.read(("Spectral Cubes/"+Cube_Name_Load_p))[:,crop[1][0]:crop[1][1],crop[0][0]:crop[0][1]]
                mask = scW.mask.include() # Need to create a mask because it doesn't get splcied
                
            else:
                if i == istart+1:
                    scW = np.concatenate((scW[:].hdu.data,sc[:].hdu.data),dtype = type(sc))
                    mask = np.concatenate((mask[:],sc[:].mask.include()),dtype = type(sc[:].mask.include()))
                else:
                    scW = np.concatenate((scW[:],sc.hdu.data[:]),dtype = type(sc))
                    mask = np.concatenate((mask[:],sc[:].mask.include()),dtype = type(sc[:].mask.include()))
            scUNIT=copy.deepcopy(sc[0][0][0].unit)
            del sc
        except Exception as e:
            print(e)
            print("-"*60)
            traceback.print_exc(file=sys.stdout)
            missed=True
            if hit:
                break
            
    # This only matters for formatting reasons:
    def duh(lol):
        gp = np.where(lol!=np.nan)
        lol[gp]=True
        return lol # Anywhere that has data will be unmasked
    
    reheader["NAXIS3"] = len(scW)
    Full_Mask = LazyMask(function = duh,data = mask, wcs = rewcs)
    
    
    scWsc = SpectralCube(data = scW,wcs = rewcs, header = reheader, mask = Full_Mask)# The spliced cube

    scWsc.allow_huge_operations=True
    scWsc = scWsc*scUNIT#Add unit back in

    if Save !='':
        scWsc.write("Spectral Cubes/"+Save,overwrite=True)
    else:
        scWsc.write("Spectral Cubes/"+Cube_Name_Save,overwrite=True)
    del scWsc
    del Full_Mask
    del reheader
    del scW
    del rewcs
    gc.collect()



In [ ]:
###
# Velocity reprojection
###
        
# Smooth the data (gaussian) to match the velocity resolution of other cubes

def Repo_Velocity(scp,Cube_Name_Save,Cube_Information,vel_range=251*u.km/u.s):
    
    print("Start velocity reprojection of" ,Cube_Name_Save)
    
    vel_prime = Cube_Information["desired_velocity_resolution"] 
    target_velocity = Cube_Information["target_velocity"]
    Initial_vel = u.Quantity(str(np.abs(scp.header['cdelt3']))+scp.header['cunit3']) # The current velocity resolution
    restfreq = Pointing_Information['restfreq']
    
    # Gaussian width = sqrt( new_vel^2 -  old_vel^2)
    
    G_width = np.sqrt((vel_prime.to(u.km/u.s)).value**2-(Initial_vel.to(u.km/u.s)).value**2) 
    
    vel = np.arange((target_velocity - vel_range).value, (target_velocity + vel_range).value,vel_prime.to(u.km/u.s).value)*u.km/u.s #make the new velocity axis
    scWsc_copy = scp
    
    # the factor converting the gaussian width to its FWHM
    fwhm_factor = np.sqrt(8*np.log(2))

    scWsc_copy = scWsc_copy.with_spectral_unit(u.km / u.s, velocity_convention='optical', rest_value=restfreq) # Make sure it has the right rest frequency
    
    # The reprojected velocity must be larger than the initial velocity or else this gives an error
    if (G_width>0):

        scWsc_copy = scWsc_copy.spectral_smooth(Gaussian1DKernel(G_width/fwhm_factor))#Preserves information from the pixels lost in downsampling
    else:
        pass
    
    print("Smoothed to Gaussian Kernel of width",G_width/fwhm_factor)

    scWsc_copy = scWsc_copy.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 251 range  

    
    Cube_Name_Save_new = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(vel_prime.value)+'_vel_res_'+'.fits'
    
    
    scWsc_copy.write("Spectral Cubes/"+Cube_Name_Save_new,overwrite=True)
    
    
    print("Wrote reprojected cube to","Spectral Cubes/"+Cube_Name_Save_new)
    
    gc.collect()
    del scWsc_copy

    gc.collect()######################################################################
    return Cube_Name_Save_new



    

In [ ]:
# after the reprojection, there may be duplicated data from the fact that it just fills in stuff 
# expecting new data to be there. This fixes it by checking to see if it matches other data, then removes
# the repeated slices. in other words, it may be set to reproject to 0-500 km/s through the velocity channels
# but there may only be data between 100-400, so it will fill it in with copied data channels. This will check for those.

# Fix reprojected repeated pixels:

# All these parameters only matter here because they are in the cube name
# Line_Name = Line Name, Beam_Size=beam size, FOV=field of view, min_vel =  velocity resolution

def Remove_Repeated_Pixels(Cube_Name_Load,crop_nans=False):
    
    Cube_Name_Save = "Cropped_"+Cube_Name_Load

    scRRP = SpectralCube.read("Spectral Cubes/"+Cube_Name_Load).with_spectral_unit(u.km/u.s,velocity_convention="radio")
    scRRP.allow_huge_operations=True
    
    sp=0 #starting pixel with real data (tbd)
    
    for lmi in range(len(scRRP)):
        #Check to see if the slice has been repeated by the interpolation function
        if(np.round(np.nanmean(scRRP[lmi].hdu.data),9)==np.round(np.nanmean(scRRP[lmi+1].hdu.data),9)):

            sp = lmi+1
        else:
            print("Good data starting from channel",sp,"; start has been cropped to that channel")
            break
            
    l = len(scRRP)-1
    ep=l
    for lmi in range(l):
        #Cehck again, starting from the end this time
        if(np.round(np.nanmean(scRRP[l-lmi].hdu.data),9)==np.round(np.nanmean(scRRP[l-lmi-1].hdu.data),9)):
            ep = l-lmi-1

        else:
            print("Good data ending at channel",ep,"; end has been cropped to that channel")
            break

    #sp,ep, These are the start and stop slices where the actual unique data resides

    scRRP.allow_huge_operations=True
    scRRP = scRRP[sp:ep]

    if (crop_nans):
        center = scRRP.wcs[:][:][0].pixel_to_world(0,0)
        print("Cropping nans...")
        scRRP = Crop_Around_Center(scRRP,0*u.deg,center=center,desired_fov=[99999999999,99999999999999]*u.pc,distance=1000*u.pc)


    scP_Hdu=scRRP.hdu
    zeros=((scP_Hdu.data[:,:,:]==0))
    bp = np.where(zeros)
    scP_Hdu.data[bp]=np.nan
    scRRP = SpectralCube.read(scP_Hdu)


    scRRP.allow_huge_operations=True
    #print(scRRP.beam)
    #print(scRRP.spectral_axis)
    #print("Beam:", scRRP.beam)
    #print("Unit:", scRRP.unit)

    scRRP = scRRP.to(u.K)

    scRRP.write(("Spectral Cubes/"+Cube_Name_Save),overwrite=True)
    del scRRP
    del scP_Hdu

    print("Cropped cube saved as ","Spectral Cubes/"+Cube_Name_Save)

# Dendrogram Calculation

In [ ]:

# d = astrodendro.Dendrogram.compute(datn,min_delta=m*delt_factor,min_value=m*5*(noise_factor),min_npix=beam_area_ratio.value*pix_thresh_factor)

# min_delta = how many times the pixel brightness must be (multiplied by the noise) before a pixel is considered outside the current structure.
# noise_requirement = how many times the noise we consider before allowing pixels
# Minimum_Pixel_Requirement = how many pixels must there be before we call it a structure. I use the beam_area_ratio for this

def Dendrogram_Calculation(Cube_Information,Minimum_Pixel_Requirement=1,min_delta=5,noise_requirement=5,Dendrogram_Save_Name='',manual_noise=0*u.K,image_args={},parts=[0]):

    Line_Name = Cube_Information['File_Descriptor']
    File_Name = Cube_Information['File_Name']
    distance = Cube_Information['distance']
    vel_prime = Cube_Information['velocity_scale']
    center = Cube_Information['center']
    Prime_Beam = (Cube_Information['beam_minor'].to(u.rad)*distance / u.rad).to(u.pc) #The current beam size in pc
    pathCont = Cube_Information["Corresponding_Continuum"]
    beam_area_ratio = Cube_Information['beam_area_ratio']
    
    Cube_Information["Dendrogram_Noise_Threshold"] =noise_requirement
    Cube_Information["Minimum_Pixel_Requirement"] =Minimum_Pixel_Requirement
    Cube_Information["min_delta"] =min_delta

    ###### 
    ### prepare some data along with the dendrogram so we can better understand what we're looking at
    ######
    
    # Load The SC
    if parts ==[0]:
        Qp = SpectralCube.read("Spectral Cubes/"+File_Name).with_spectral_unit(u.km/u.s,velocity_convention="radio") 
    else:
        Qp = SpectralCube.read("Spectral Cubes/"+File_Name).with_spectral_unit(u.km/u.s,velocity_convention="radio")[parts[0]:parts[1],parts[2]:parts[3],parts[4]:parts[5]]
    Qp.allow_huge_operations=True

    Q = Qp.to(u.K)# Jy to Kelvin in case its not already in
    sc = Q.unmasked_copy() # remove a mask if there is one 
    del Q 
    del Qp

    # The cube data

    scW = sc.wcs[:][:][0]
    datn = sc.hdu.data
    moment_0_sc  = sc.moment(order=0,how='slice')            # Calculate the Moment 0 map to use for plotting
    moment_0_sc.write(("Spectral Cubes/"+Cube_Information["File_Name"][0:len(Cube_Information["File_Name"])-5]+"_Moment_0"+".fits"),overwrite=True)
    header = sc.hdu.header
    
    #####
    #####
    # Reproject Continuum image to match the current image (this doesnt require any other data reduction steps since we already did those to the current SC)
    #####
    #####
    
    # Continuum image, for use in the column density maps
    
    scCont = spectral_cube.Projection.from_hdu(fits.open("Spectral Cubes/"+pathCont)) # {Why was there a  [0] here?}
    
    try:
        BUNIT = scCont.unit
        print("The continuum has unit",BUNIT)
        scCont = scCont.to(u.Jy/u.beam) #Fails if unitless
        BUNIT = scCont.unit
        print("Change to:",BUNIT)
        print("verify unit length is non zero, length = ",len(str(BUNIT)),str(BUNIT)) #Fails if there is no unit

    except:
        print("The continuum is unitless; unit defaults to Jy/beam")
        scCont = scCont*u.Jy/u.beam #Some data was taken before BUNIT were implemented, but they are in Jy/beam
        
    Use_Dict = {'desired_beam_oversampling': "NA"} # this is irrelevant since the continuum is higher resolution than the data cubes
    Use_Dict["distance"] = distance # same as for the data cube
    
    Continuum_Information = Update_Cube_Information(Use_Dict,pathCont)
    
    
    ## Reproject continuum 
    scCont.allow_huge_operations=True
    Continuum = scCont.reproject(moment_0_sc.header)
    del scCont
    Continuum_Data  = Continuum.hdu.data
    Cont_Save_File=(Continuum_Information["File_Name"][0:len(Continuum_Information["File_Name"])-5]+"_Reproject_to_"+Cube_Information['File_Descriptor']+".fits")
    
    # save the reprojected continuum     
    Continuum.write(("Spectral Cubes/"+Cont_Save_File),overwrite=True)
    Continuum_Information = Update_Cube_Information(Continuum_Information,Cont_Save_File)
    Cube_Information["Corresponding_Continuum_Reprojected"] = Cont_Save_File
    
    # calculate the flux based on the beam to find te column density
    # flux = Jansky/beam/(pixels/beam)/(beam_physical_size in pc^2)*particle number density (using a factor to account for the other, non H2 particles in the 850um continuum)
    pc_per_pixel_continuum = Cube_Information['spatial_scale_y']

    # The flux, converting from Jy/beam, accounting for the beams physical size
    flux = (Continuum_Data*Continuum[0][0].unit)/((Continuum_Information["beam_area_ratio"]*(pc_per_pixel_continuum**2)).to(u.cm**2/u.beam))
    
    Column_Densities = Flux_to_Mass(flux,distance)*Num_per_kg  # Calculates the number-luminosity from the particle flux and converts it to mass using the conversion factor from Bolatto 2013a, in number/cm^2
    
    # Ratio of flux / column density (compare the continuum to teh line emission)
    rm=moment_0_sc.hdu.data/Column_Densities
    rmU = rm*moment_0_sc[0][0].unit # Just put the units back in
    del rm
    
    #rmU = np.array(rmU/10**22,dtype='float64')# reduce to floating point scale because they cant handle nubers this big

    ###### there are errors at low points, giving negative mass, so i exclude them if theyre below the noise

    bp = np.where(Column_Densities<=7*10**22/u.cm**2)
    bp2 = np.where(moment_0_sc.hdu.data < (np.nanstd(moment_0_sc.hdu.data,where= ((moment_0_sc.hdu.data>0)  | (moment_0_sc.hdu.data<0) )))) # below the Noise (K km/s)
    Continuum_Data_Cropped = copy.deepcopy(Continuum_Data)
    Continuum_Data_Cropped[bp] = np.nan # A version of the continuum data withuot the negative pixels
    
    Column_Densities_Cropped = copy.deepcopy(Column_Densities)
    Column_Densities_Cropped[bp]=np.nan # Remove negative pixels from the column densities
    
    rmU[bp]=np.nan # Remove the negative pixels from either the moment 0 or the continuum
    rmU[bp2]=np.nan

    #######
    #######
    # Calculate the RMS noise of the cube (for the line)
    #######
    #######

    Non_nan=((datn[0]>0)  | (datn[0]<0 )) # All the data that is not a nan value in the first (emissionless) channel of the cube

    m = np.nanstd(datn[0],where= Non_nan)*sc[0][0][0].unit #Noise (K)
    if np.sum(((datn[0]>0)  | (datn[0]<0 )))<100:
        #check to see if there is actually data at this slice. if not, jump ahead
        Non_nan=((datn[32]>0)  | (datn[32]<0 )) # All the data that is not a nan value in the first (emissionless) channel of the cube
        m = np.nanstd(datn[32],where= Non_nan)*sc[0][0][0].unit #Noise (K)
        print("Noise slice=",32)
    elif np.sum(((datn[0]>0)  | (datn[0]<0 )))<1000:
        Non_nan=((datn[20]>0)  | (datn[20]<0 )) # All the data that is not a nan value in the first (emissionless) channel of the cube
        m = np.nanstd(datn[20],where= Non_nan)*sc[0][0][0].unit #Noise (K)
        print("Noise slice=",20)
        
        
    if manual_noise != 0*u.K:
        m = manual_noise
    Cube_Information["RMS_Noise"]=m
    
    print(m,"  RMS Noise for",Cube_Information["File_Descriptor"])
    
 





    ######
    ######
    ######
    ### Generate relevant plots before the dendrogram so we know what we're looking at
    ######
    ######
    ######
        
    #def Make_Image_Plot(Title,Cube,vmin,vmax,rows=1,columns=1,index=1,show=False):

    ###### Moment 0 and continuum plots
    show=True
    for key in image_args.keys():
        if key =="SHOW":
            show=image_args[key]

    Make_Image_Plot((Line_Name+" Moment 0"),"Moment 0 Intensity (K km s^-1)",moment_0_sc.hdu.data,wcs=moment_0_sc.wcs,vmin=0,vmax=np.nanmax(moment_0_sc.hdu.data),rows=1,columns=1,index=1,show=False,Cube_Info=Cube_Information,image_args=image_args)
    
    if show:
        pylab.show()

    Make_Image_Plot((Line_Name+" Moment 0 PC"),"Moment 0 Intensity (K km s^-1)",moment_0_sc.hdu.data,wcs=moment_0_sc.wcs,vmin=0,vmax=np.nanmax(moment_0_sc.hdu.data),rows=1,columns=1,index=1,show=False,Axis="PC",Cube_Info=Cube_Information,image_args=image_args)
    
    Center_p = moment_0_sc.wcs.world_to_pixel(center)
    pylab.annotate(text="Center",fontsize=12,xy=(Center_p[0]+20, Center_p[1]+10), rotation=-20,color="gray")  
    pylab.annotate(text="x",fontsize=12,xy=(Center_p[0], Center_p[1]), rotation=-10,color="gray")  
    if show:
        pylab.show()

    
    Make_Image_Plot((Line_Name+" Moment 0 PC"),"Moment 0 Intensity (K km s^-1)",moment_0_sc.hdu.data,wcs=moment_0_sc.wcs,vmin=0,vmax=np.nanmax(moment_0_sc.hdu.data),rows=1,columns=1,index=1,show=False,Axis="PC",Cube_Info=Cube_Information,image_args=image_args)
    
    Center_p = moment_0_sc.wcs.world_to_pixel(center)
    pylab.annotate(text="Center",fontsize=12,xy=(Center_p[0]+20, Center_p[1]+10), rotation=-25,color="gray")  
    pylab.annotate(text="x",fontsize=12,xy=(Center_p[0], Center_p[1]), rotation=-10,color="gray")  
    if show:
        pylab.show()

    
    Make_Image_Plot((Line_Name+"Column Density"),"Column Density (# cm^-2)",Column_Densities,wcs=moment_0_sc.wcs,vmin=float(np.nanmin(Column_Densities.value)),vmax=float(np.nanmax(Column_Densities.value)),rows=1,columns=1,index=1,show=False,Axis="PC",Cube_Info=Cube_Information) # Use the full line SC information here becasue I dont need specific things
    
    pylab.annotate(text="Center",fontsize=12,xy=(Center_p[0]+20, Center_p[1]+10), rotation=-20,color="gray")  
    pylab.annotate(text="x",fontsize=12,xy=(Center_p[0], Center_p[1]), rotation=-10,color="gray")  
    if show:
        pylab.show()


    Make_Image_Plot((Line_Name+" Moment 0 over Column Density of the Continuum"), "Ratio (K km s^-1 over (# cm^-2))",rmU.value,wcs=moment_0_sc.wcs,vmin=np.nanmean(rmU.value)*.5,vmax=abs(np.nanmean(rmU.value))*8,rows=1,columns=1,index=1,show=False,Axis="PC",Cube_Info=Cube_Information)
    
    pylab.annotate(text="Center",fontsize=12,xy=(Center_p[0]+20, Center_p[1]+10), rotation=-20,color="gray")  
    pylab.annotate(text="x",fontsize=12,xy=(Center_p[0], Center_p[1]), rotation=-10,color="gray")  
    if show:
        pylab.show()

    
    
    
    
    
    
        
    ############
    # Calculate the dendrogram 3d image of all the structures in the data set, based on the pixel, noise, and delta parameters
    ############
    
    Delta_Factor = m.value*min_delta # A multiple of the noise, this is how much a pixel's brightness must differ before it is considered separate from teh current structure
    min_value = m.value*noise_requirement # The minimum value of a structure. If this is positive, then only emission will be consdered.
    min_npix = beam_area_ratio.value*Minimum_Pixel_Requirement # How many pixels before a structure is resolved. I say they must have at least enough pixels to fill the beam so tehy are not just artifacts.
    
    print("Computing Dendrogram...")
    
    Dendrogram = astrodendro.Dendrogram.compute(datn,min_delta=Delta_Factor,min_value=min_value,min_npix=min_npix) 
    
    
    Save_Dend=Line_Name+"_"+File_Name[0:len(File_Name)-5]+"_Dendrogram"
    
    
    if(Dendrogram_Save_Name==''):
        Dendrogram.save_to(("Dendrograms/"+Save_Dend),"fits")
    else:
        Dendrogram.save_to(("Dendrograms/"+Dendrogram_Save_Name),"fits")

    Cube_Information["Corresponding_Dendrogram"] = Save_Dend
    
    # Show the dendrogram
    p1=Dendrogram.plotter() 
    v1 = Dendrogram.viewer()
    
    v1.show()
        
    print("Dendrogram for",Cube_Information["File_Descriptor"],"saved to","Dendrograms/"+Save_Dend+".fits")



# PPV statistics

 This function uses the dendrogram as an input and calculates all the quantities
 such as size, linewidth, Column Density, RMS velocity (velocity dispersion, a measure 
 of turbulence), Luminosity (using the distance and the flux), Distance to structure (only
 matters for the CMZ), the Moment0_Flux, and the V_RMS_error

 
 These are all the things calculated in the Shetty 2012 paper

 To exclude incomplete, unresolvable, or otherwise nan strucutres, we use some exclusion properties:
 
 1. Don't allow any structure under 3 pc in size (the smallest beam size) {now the min size is zero because i set the pixel requirement higher, but it used to be this}
 
 2. Don't allow any structure over 18 pc
 
 3. Don't allow structures that are only visible in 2d
 
 4. In the dendrogram calculation, there are additional restrictions placed on the amount of minimum pixels in a stuctures and the noise, and     noise-delta. I use minimum 5 sigma noise, 5 sigma as delta, and require there to be enough pixels to fill a whole beam (the beam_area_ratio).


Returns np.array(SizeA), np.array(SigmaA), np.array(CDA) ,np.array(LuminA) ,np.array(SIDS) , np.array(MOM0_FLUX) , np.array(Distances), np.array(V_rms_err) for each structure in the dendrogram and returns them in the form [[][]], where [Leaves][Branches] are the different kinds of structures in the dendrogram



In [ ]:

#Continuum is in Jansky/Beam, Line data should have the unit specified in the metadata as 'data_unit'

#Dendrogram=the computed dendorgam, 
#Cube = the SC associated with the dendrogram
#Cube_Information = defined above
#Max_Size=the maximum allowed size for a strcutre in u.pc
#Min_size = the minthra-carta jk it means the minimum size for a structure in u.pc hahahahahaha can you tell its 2 am right now?
#Trunks= True/False if you want/dont want to include the dendrogram trunks as branches
#edge_cases = accept structures that seem to go outside the cube? True/False
#Distance calculation= for the CMZ, the distance from the center may be a significant portion of the 8170 distance. I created a function to calculate teh aproximate distance


# Note: The column density information is not yet fully bug tested, and it relies on continuum data that came from Bolatto 2013a, and a conversion factor that must be estimated; 
# do not use it as gospel
                        
def Dendrogram_Stats(Dendrogram,Cube,Continuum_Cube,Continuum_Information,Cube_Information,Trunks=True,min_size=0*u.pc,max_size=18*u.pc,edge_cases=False,distance_calculation=False,r_err=False):
    
    #initial the statistacs to return, in [[leaves], [branches]]format
    Size,RMS_Velocity,V_rms_err,Luminosity,CDs,SIDs,MOM0_FLUX,Distances = [[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]]
    R_errs=[[],[]]
    #Size,RMS_Velocity,V_rms_err,Luminosity,CDs,SIDs,MOM0_FLUX,Distances = np.array([[],[],[],[]],dtype=type(1*u.pix)),np.array([[],[],[],[]],dtype=type(1*u.pix)),np.array([[],[]],dtype=type(1*u.pix)),np.array([[],[]],dtype=type(1*u.pix)),np.array([[],[]],dtype=type(1*u.pix)),np.array([[],[]],dtype=type(1*u.pix)),np.array([[],[]],dtype=type(1*u.pix)),np.array([[],[]],dtype=type(1*u.pix))

    dist_val=Cube_Information['distance']  #convert to the value in pc
    distance = dist_val
    center = Cube_Information["center"]
    m = Cube_Information["RMS_Noise"]
    d_copy = Dendrogram #make on that i can edit 
        
    center_ra_pix,center_dec_pix = int(Cube.wcs[:][:][0].world_to_pixel(center)[0]),int(Cube.wcs[:][:][0].world_to_pixel(center)[1])
    
    LineData = Cube.hdu.data
    sliced= LineData[int(len(LineData)/2)]
    CubeShape = np.shape(sliced) #the shape of a slice of the cube
    
    ContData = Continuum_Cube.hdu.data
    
    
    # get properties for the moment 2 map
    cube = Cube #copy
    chans,ny,nx = cube.hdu.data.shape
    bunit = u.Unit(cube.header['bunit'])
    cunit = u.Unit(cube.header['cunit3'])
    cdelt = cube.header['cdelt3']*cunit
    crval = cube.header['crval3']*cunit
    crpix = cube.header['crpix3']
    pix_cen = np.array([((i+crpix-1)*cdelt+crval).value for i in np.arange(chans)])*cunit
    pix_cen = np.array([np.full((ny,nx), cen) for cen in pix_cen])*pix_cen.unit

    # mask cube
    min_snr = Cube_Information["Dendrogram_Noise_Threshold"]            # check dendrogram threshold 
    m=Cube_Information["RMS_Noise"]
    cube.hdu.data[cube.hdu.data*bunit<min_snr*m] = np.nan


    


    #Find the part of the cube that actually has data (useless if the nans are cropped but i have this anyway)
    
    DataShape=[[0,0],[0,0]]

    for lmi in range(CubeShape[0]):
        allData=np.nansum(sliced[lmi])
        if(allData>0 or allData<0):
            DataShape[0][0] = lmi+1 # teh left edge
            break
    for lmi in range(CubeShape[0]):
        allData=np.nansum(sliced[CubeShape[0] - lmi -1])
        if(allData>0 or allData<0):
            DataShape[0][1] = CubeShape[0] - lmi -1 #the right edge
            break
    for lmi in range(CubeShape[1]):
        allData=np.nansum(sliced[:,lmi])
        if(allData>0 or allData<0):
            DataShape[1][0] = lmi+1 #the bottom
            break
    for lmi in range(CubeShape[1]):
        allData=np.nansum(sliced[:,CubeShape[1] - lmi -1])
        if(allData>0 or allData<0):
            DataShape[1][1] = CubeShape[1] - lmi -1 #the top
            break
            
            
    # Go through every structure identivfied
    
    for t in Dendrogram.all_structures: 

        I = t.indices() #the pixel values in x, y of the structure
        
        
                
        Cont = True
        if t.is_branch:
            
            # If the structure is a branch with no parents, it is a trunk
                if t.parent==None:
                    
                    if(Trunks):
                        Cont = True
                    else:
                        Cont = False
                else:
                    Cont=True
        
        # Exclude structures that go off the edge of the data and are therefore not fully resolved
        
        if edge_cases == False:
            
            #For non-rotated data cubes, it is really easy to see if a structure is on the edge of the cube
            
            if Cube_Information['target_image_rotation'] == 0*u.deg :
            
            
                for lmi in range(len(I[0])):
                        
                    # cehck if there are any nan values nearby or any edges of the cube nearby
                    
                    if(I[1][lmi] <= DataShape[0][0] or I[1][lmi] >= DataShape[0][1] or I[2][lmi] <= DataShape[1][0] or I[2][lmi] >= DataShape[1][1]):
                        #print(I[1][lmi], DataShape[0][0], I[1][lmi] , DataShape[0][1] , I[2][lmi],DataShape[1][0], I[2][lmi],DataShape[1][1])
                        #print(I[1][lmi] <= DataShape[0][0], I[1][lmi] >= DataShape[0][1] , I[2][lmi] <= DataShape[1][0] , I[2][lmi] >= DataShape[1][1],I,"A")
                        Cont=False
                        break
                        
                    # Check if they are ON the edge also
                    
                    elif(I[1][lmi] <= 1 or I[1][lmi] >= CubeShape[0] or I[2][lmi] <= 1 or I[2][lmi] >= CubeShape[1]):
                        #print(I[1][lmi] <= 1 or I[1][lmi] >= CubeShape[0] or I[2][lmi] <= 1 or I[2][lmi] >= CubeShape[1],"B")
                        Cont=False
                        break
                        
            else:          
            
            #For rotated cubes, I check every direction to make sure there are no nan values beside them
            
                for lmi in range(len(I[0])):
                    NansNE=0
                    NansSE=0
                    NansNW=0
                    NansSW=0
                    Length = 3
                    for lmj in range(Length):
                        #Check four 45 degree prongs from each point and see if they have nans. If that happens its too close to the boundary or the data is bad
                        try:

                            if(sliced[I[1][lmi]+lmj,I[2][lmi]-lmj]>0 or sliced[I[1][lmi]+lmj,I[2][lmi]-lmj]<0 ):
                                pass
                            else:
                                NansSE+=1
                            if(sliced[I[1][lmi]-lmj,I[2][lmi]-lmj]>0 or sliced[I[1][lmi]-lmj,I[2][lmi]-lmj]<0 ):
                                pass
                            else:
                                NansSW+=1
                            if(sliced[I[1][lmi]-lmj,I[2][lmi]+lmj]>0 or sliced[I[1][lmi]-lmj,I[2][lmi]+lmj]<0 ):
                                pass
                            else:
                                NansNW+=1
                            if(sliced[I[1][lmi]+lmj,I[2][lmi]+lmj]>0 or sliced[I[1][lmi]+lmj,I[2][lmi]+lmj]<0 ):
                                pass
                            else:
                                NansNE+=1
                        except:
                            #only fails if the I goes close to the boundary of the cube and tries to get a pixel outside the cube
                            Cont = False
                            break
                    #count the number of nans nearby:
                        
                    if(NansNE>2 or NansNW>2 or NansSE>2 or NansSW>2):
                        Cont=False
                        break

        # Assuming the structure is not an error, continue:
        
        if(Cont):
            
            #The metadata is used to calculate various properties. it passes the spatial scale, data unit, and such, for Jy->K calculation
            metadata=Cube_Information
            
            # calcuate the properties of the using this function from the astrodendro library
            s = PPVStatistic(t,metadata=metadata) 
            
            s_radius = s.radius #Give the size in degrees
            s_v_rms = s.v_rms #in u.km/u.s
            pc_per_pixel = metadata['spatial_scale_x']
            #Parsec_Size = (float(s_radius.to(u.rad).value*dist_val)) # Convert to parsecs using the distance in Pc
            # Parsec_Size = (s_radius*u.pix**2*pc_per_pixel).to(u.pc) # If needed, Convert to parsecs using the spatial scale in Pc/pixel
            Parsec_Size = ((s.radius).to(u.rad)*dist_val/u.rad).to(u.pc) # convert to pc instead of deg/pixel
            #also check to make sure the size is greater than the minimum size (1/3 the beam) any less and it will be too much noise
            #and make sure the rms velocity is non-zero. otherwise, its not a 3d structure
            
            if(Parsec_Size<max_size and Parsec_Size>min_size and s_v_rms>.01*u.km/u.s):
            
                '''unused
                # A slightly different radius calculation, using the mean-projected area . from shetty 2012
                nproj_pix=len(set(zip(*tuple(I[i] for i in [1,2]))))
                #Intensity mean-weighted velocity:
                v_IWM = np.nansum(LineData[I]*(DataVel[I[0]])/u.km*u.s)/np.nansum(LineData[I])
                # calculating the Rms velocity manually
                sig_Sh = np.sqrt(np.nansum(LineData[I]*((DataVel[I[0]])/u.km*u.s-v_IWM)**2)/np.nansum(LineData[I])) 
                '''
                
                #column density
                #Need the flux from the continuum
                #Convert to Jansky from Jansky per beam:

                Cont_Flux=0

                proj = tuple(set(zip(*tuple(I[i] for i in [1,2]))))
                for lmi in range(len(proj)):

                    Cont_Flux+=ContData[proj[lmi]]
                Cont_Flux = Cont_Flux*Continuum_Cube[0][0].unit # add the unit back in
                
                # assuming the continuum is reprojected to the cube, i can use the beam_area_ratio of the cube here:
                # Convert from Jansky / beam. The beam is changed from FWHM to Gaussian
                #Cont_Flux=Cont_Flux/(((metadata['beam_area_ratio']*(pc_per_pixel**2)).to(u.cm**2/u.beam))*(2*np.sqrt(2*np.log(2))))
                
                Cont_Flux=Cont_Flux/(((metadata['beam_area_ratio']*(pc_per_pixel**2)).to(u.cm**2/u.beam)))
                 
                #Fro FWHM beam:
                Dust_Column = Flux_to_Mass(Cont_Flux,distance)*Num_per_kg   # calculate column density, from flux to mass

                #check for divide by zero errors:
                if(str(Dust_Column) == str(np.nan) or str(Dust_Column)==str(np.inf)):
                    Dust_Column=0
                lum = Flux_to_Lum(s.flux,distance)
                s_flux = s.flux

                Index = tuple(I[i] for i in [0,1,2])
                K_Km_s_Flux=np.nansum(LineData[Index]*metadata["velocity_scale"]*Cube[0][0][0].unit)#Find the total flux from the structures in K km/s, assuming the input data is in K as it should be, 
                
                if distance_calculation:
                    Distance = np.sqrt((float(s.x_cen/u.pix)-center_ra_pix)**2+(float(s.y_cen/u.pix)- center_dec_pix)**2).to(u.rad)*metadata['spatial_scale']*dist_val#pc dist from barycenter
                else:
                    Distance = np.nan
                
                NF=5
                iterations=5
                #V_err= Get_V_rms_err(dend1=d_copy,idx=int(t.idx),struct=t,m=m.value,NF=NF,iterations=iterations,metadata=metadata)
                #Get V rms error by taking the 84th and 16th percentile of the second moment of the structure.
                '''
                flat_pixels=[] #get the 2 projection fo the structure:
                for lmi in range(len(I[0])):
                    if [I[lmi][0] ,  I[lmi][1]] not in flat_pixels:
                        flat_pixels.append([I[lmi][0] ,  I[lmi][1]])
                '''
                #C1 = copy.deepcopy(Cube[I]) #get the data from the structure
                #mom2 = C1.moment2()
                

                
                
                #velocity_scale = metadata['velocity_scale']

                #dv = velocity_scale
                '''
                dv = s.velocity_scale if s.velocity_scale is not None else u.pixel
                ax = [0, 0, 0]
                ax[s.vaxis] = 1
                
                test =  dv * np.sqrt(s.stat.mom2_along(tuple(ax)))
                mom2 = s.stat.mom2()
                #print(mom2)
                
                w = np.atleast_2d(tuple(ax)).astype(float)
                for row in w:
                    row /= np.linalg.norm(row)
                #print(np.dot(w, mom2))
                result = np.dot(np.dot(w, mom2), w.T)
                #print(result,dv * np.sqrt(result))
                
                #mean/median of per pixel moment 2
                ###################################################################################################

                mean = np.nanmean(mom2)
                std  = np.nanstd(mom2)
                p16,median,p84 = np.nanpercentile(mom2, (16,50,84))
                
                print("1",mean,s_v_rms,p16,median,p84,test)
                #8.320892732487192 9.807691033727796 km / s 8.320892732487192 8.320892732487192 8.320892732487192 9.807691033727796 km / s
                #4.632530923209886 9.807691033727796 km / s -5.917875023908338 6.736166347151977 13.60092186257982 9.807691033727796 km / s
                #9.049809177357027 9.807691033727796 km / s 3.9121341645280565 7.12497664499009 13.60092186257982 9.807691033727796 km / s

                V_err = std
                
                
                '''
                
                

                mask    = t.get_mask() #get the pixel values from the structure
                mcube   = np.nan_to_num(cube.hdu.data*mask)*bunit #Make a cube containing only the structure

                mom0 = moment0(mcube,             cdelt)
                mom1 = moment1(mcube, mom0,       cdelt, pix_cen)
                mom2 = moment2(mcube, mom0, mom1, cdelt, pix_cen)

                mean = np.nanmean(mom2.value)
                std  = np.nanstd(mom2.value)
                p16,median,p84 = np.nanpercentile(mom2.value, (16,50,84))
                
                #print("2",mean,s_v_rms,p16,median,p84,std)
                
                V_err = std*u.km/u.s

                ###################################################################################################3
                if r_err:
                    R_err= (Get_R_err(dend1=d_copy,idx=int(t.idx),struct=t,m=m.value,NF=NF,iterations=iterations,metadata=metadata).to(u.rad)*dist_val/u.rad).to(u.pc) # convert to pc instead of deg/pixel
                else:
                    R_err = 0
                
                
                if(t.is_leaf):
                    # add teh value to the leaf axis: 
                    
                    
                    Size[0].append(Parsec_Size)
                    RMS_Velocity[0].append(s_v_rms)
                    V_rms_err[0].append(V_err) 
                    Luminosity[0].append(lum) 
                    CDs[0].append(Dust_Column) 
                    SIDs[0].append(t.idx) 
                    MOM0_FLUX[0].append(K_Km_s_Flux) 
                    Distances[0].append(Distance)
                    R_errs[0].append(R_err)
                

                    
                if(t.is_branch	):
                    # add teh value to the branch axis: 
                    
                    
                    Size[1].append(Parsec_Size)
                    RMS_Velocity[1].append(s_v_rms)
                    V_rms_err[1].append(V_err) 
                    Luminosity[1].append(lum) 
                    CDs[1].append(Dust_Column) 
                    SIDs[1].append(t.idx) 
                    MOM0_FLUX[1].append(K_Km_s_Flux) 
                    Distances[1].append(Distance)
                    R_errs[1].append(R_err)
                    
                    
                del s
                    
                    
    if r_err:
        return Size,RMS_Velocity,V_rms_err,Luminosity,CDs,SIDs,MOM0_FLUX,Distances,R_errs
    else:
        return Size,RMS_Velocity,V_rms_err,Luminosity,CDs,SIDs,MOM0_FLUX,Distances

                        
                        
 


# Useful functions

In [ ]:


# This is a mass conversion factor for CO 3-2 to H2 calibrated using the 850 Ghz dust continuum
# It will not be used widly, but if it was I would need a relation factor for each molecular based 
# on their relative abundance, and their line transition ratio.
# This has also been divided by two to account for the higher metallicity of NGC253 as compared to the CMZ
# as Krieger did in 2020

a_850 = 6.7*10**19*u.erg/u.s/u.Hz/u.M_sun #6.7+-1.7, from Bolatto 2013a

a_850 = 6.7*10**19*u.erg/u.s/u.Hz/(1*u.M_sun).to(u.kg) #6.7+-1.7, from Bolatto 2013a


def Flux_to_Mass(flux, dist, Lum_per_mass_factor=a_850):
    
    #Here is the manual process to convert to ergs
    J_to_e = 10**-23*u.erg/u.s/u.cm**2/u.Hz/u.Jy #Jansky to flux in erg/(s cm^2 Hz)
    flux_erg = flux*J_to_e
    #flux_erg = flux.to(u.erg/u.cm**2/u.s/u.Hz) # doesnt work
    #here is the conversion factor for Mpc to cm
    #Mpc_to_cm = 3.086*10**24 * u.cm/u.Mpc
    dist_cm=dist.to(u.cm)
    
    # Now use the distance and flux to calculate the luminosity, then use that and the conversion factor to find the mass
    
    # L = 4pi*r2 * Flux
    
    L = 4*np.pi*(dist_cm)**2*flux_erg #Megaparsec is converted to cm
    
    
    
    
    
    M_mol = L/Lum_per_mass_factor #Just in Solar mass*1.989*10**30*u.kg/u.M_sun #Determines mass of the cont for 850 in kg
    
    return M_mol

        
# This assumes the input flux will be in Jansky, which the astrodendro library defaults to.
# And make sure the input distance is in Mpc

def Flux_to_Lum(flux, dist):
    
    #Here is the manual process to convert to ergs
    #J_to_e = 10**-23*u.erg/u.s/u.cm**2/u.Hz/u.Jy #Jansky to flux in erg/(s cm^2 Hz)
    J_to_e = 10**-23*u.erg/u.s/u.cm**2/u.Hz/u.Jy #Jansky to flux in erg/(s cm^2 Hz)

    flux_erg = flux*J_to_e
    #flux_erg = flux.to(u.erg/u.cm**2/u.s/u.Hz) # doesnt work
    #here is the conversion factor for Mpc to cm
    #Mpc_to_cm = 3.086*10**24 * u.cm/u.Mpc
    dist_cm=dist.to(u.cm)
    
    # Now use the distance and flux to calculate the luminosity, then use that and the conversion factor to find the mass
    
    # L = 4pi*r2 * Flux
    
    L = 4*np.pi*(dist_cm)**2*flux_erg #Return the luminosity in Erg

    
    return L
'''
def Flux_to_Lum(flux, dist):
    
    #Mpc_to_cm = 3.086*10**24 * u.cm/u.Mpc
    #dist_cm=dist.to(u.cm)
    
    # Now use the distance and flux to calculate the luminosity, then use that and the conversion factor to find the mass
    # L = 4pi*r2 * Flux
    
    L = 4*np.pi*(dist)**2*flux #Return the luminosity in K km/s *pc^2

    
    return L
'''

# gets the RMS noise of a structure (estimation)
# TO do this, we apply a avariation by adding a noise-structure to the structure and finding the std of multiple of these noise-added structures (bootstrapping method)
'''
def Get_V_rms_err(mom2,dend1,struct,idx,m,NF,iterations,metadata):
    
    
    vs=[] # an array of the calculated V_RMS
    np.random.seed((99)**2*123)
    
    for llll in range(iterations):
        
        s = dend1.__getitem__(idx) # Get the structure from the dendrogram
        
        npixels = np.product(np.shape(s.values()))
        
        additional_noise = np.random.normal(0., m*NF, npixels)
        additional_noise = np.reshape(additional_noise, np.shape(s.values()))
        
        # add or subract noise to the values and calculate the v rms, them find the std of that array and
        # call that the uncertainty in v rms for a structure
        dat1P = dend1.data[s.indices()] # a copy of the data
        dend1.data[s.indices()]+= additional_noise #add teh noise
        
        s = dend1.__getitem__(idx)
        
        vs.append(float(PPVStatistic(s,metadata=metadata).v_rms.value))
        dend1.data[s.indices()]= dat1P # reset the dend data
        
        dend1.data[s.indices()]-= additional_noise # subtract the noise also
        s = dend1.__getitem__(idx)
        
        vs.append(float(PPVStatistic(s,metadata=metadata).v_rms.value))
        
        UNIT = PPVStatistic(s,metadata=metadata).v_rms.unit
        
        del s
        
    v_rms_std = np.nanstd(vs)*UNIT
    #print(vs*UNIT,v_rms_std)
    return v_rms_std


 mean/median of per pixel moment 2
###################################################################################################

for co,CO in lines.items():
    for gal,GAL in galaxies.items():

        from astropy.table.column import Column

        catalog    = dendrograms[co][gal]['catalog']

        lw = {'mean': [], 'std': [], 'median': [], '16th': [], '84th': []}
        for idx in tqdm(moms[co][gal]['mom0'].keys()):
            mom2 = moms[co][gal]['mom2'][idx]
            mean = np.nanmean(mom2.value)
            std  = np.nanstd(mom2.value)
            p16,median,p84 = np.nanpercentile(mom2.value, (16,50,84))
            lw['mean'].append(mean)
            lw['std'].append(std)
            lw['median'].append(median)
            lw['16th'].append(p16)
            lw['84th'].append(p84)
        log_lw = {k:np.log10(v) for k,v in lw.items()}

        for k in lw.keys():
            catalog.add_column( Column(name='linewidth (mom2 '+k+')',   data=lw[k],   dtype=np.float64, unit='km/s') )

        for k in log_lw.keys():
            catalog.add_column( Column(name='log linewidth (mom2 '+k+')',   data=log_lw[k],   dtype=np.float64, unit='km/s') )

'''     
# Similar bootstrapping method as the v_rm_err calculation

def Get_R_err(dend1,struct,idx,m,NF,iterations,metadata):
    
    
    rs=[] # an array of the calculated V_RMS
    np.random.seed((99)**2*123)
    
    for llll in range(iterations):
        
        s = dend1.__getitem__(idx) # Get the structure from the dendrogram
        
        npixels = np.product(np.shape(s.values()))
        
        additional_noise = np.random.normal(0., m*NF, npixels)
        additional_noise = np.reshape(additional_noise, np.shape(s.values()))
        
        # add or subract noise to the values and calculate the v rms, them find the std of that array and
        # call that the uncertainty in v rms for a structure
        dat1P = dend1.data[s.indices()] # a copy of the data
        dend1.data[s.indices()]+= additional_noise #add teh noise
        
        s = dend1.__getitem__(idx)
        
        rs.append(float(PPVStatistic(s,metadata=metadata).radius.value))
        dend1.data[s.indices()]= dat1P # reset the dend data
        
        dend1.data[s.indices()]-= additional_noise # subtract the noise also
        s = dend1.__getitem__(idx)
        
        rs.append(float(PPVStatistic(s,metadata=metadata).radius.value))
        
        UNIT = PPVStatistic(s,metadata=metadata).radius.unit
        
        del s
        
    r_std = np.nanstd(rs)*UNIT
    #print(rs*UNIT,r_std)
    return r_std

# Return a cropped cube for some ra and dec, also crops the velocity axis if needed (0 for no crop)
# cube= the spetral cube you wish to crop
# WCS = that cube's world coordinate system (.WCS)
# Np1, Np2 = the first and final pixel that don't have nan values (the left and right bounds of the cube)
# BadVel = put a number based on the amound of velocity channels that are just noise, or leave at zero if you want to keep the noise
# D2 = put True if the cube is 2D, otherwise, put False

def Crop(cube,WCS,Np1,Np2,BadVel,D2):
    NraDP1 = [int(WCS.world_to_pixel(Np1)[0]),int(WCS.world_to_pixel(Np1)[1])]
    NraDP2 = [int(WCS.world_to_pixel(Np2)[0]),int(WCS.world_to_pixel(Np2)[1])]
    if(D2==False):
        return cube[BadVel:np.shape(cube)[0]-BadVel,NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]
    if(D2==True):
        return cube[NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]

    
    
# crops rectangular nan data added during reprojection
def Crop_Nans(data):

    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(data[0,:,:])[0]):

        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            print("F",lmi)
            break
        for lmj in range(np.shape(data[0,:,:])[1]):

            if(sx==0):            
                if(np.nanmean(data[0,lmi,:])>0 or np.nanmean(data[0,lmi,:])<0):
                    sx=lmi


            if(sy==0):
                if(np.nanmean(data[0,:,lmj])>0 or np.nanmean(data[0,:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(data[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(data[0,:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])>0 or np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])<0):
                    ey=np.shape(data[0,:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break
                
    print(sx,ex,sy,ey)
    return sx,ex,sy,ey



# calculate moment maps for all structures
###################################################################################################

def moment0(cube, chanwidth, mask_zeros=True):
    mom0 = chanwidth*np.nansum(cube.value, axis=0)*cube.unit
    if mask_zeros:
        mom0[mom0==0.0] = np.nan
    return mom0

def moment1(cube, moment0, chanwidth, pix_centers):
    mom1 = np.nansum(cube*pix_centers*chanwidth, axis=0)/moment0
    return mom1




def moment2(cube, moment0, moment1, chanwidth, pix_centers, mask_zeros=True):
    mom2 = np.full_like(moment1.value, 0.0) *cube.unit*pix_centers.unit**2*chanwidth.unit
    for chan in np.arange(cube.shape[0]):
        mom2 += cube[chan] *(pix_centers[chan]-moment1)**2 *chanwidth
    mom2 = np.sqrt(mom2/moment0)
    if mask_zeros:
        mom2[mom2==0.0] = np.nan
    return mom2



# Plotting

In [ ]:

#Make an image from the SC data

#Data=the image data to plot
#vmin,vmax=the min/max scale for the graph's color bar
#Name2, another descriptor

def Make_Image_Plot(Title,Units,Cube_data,wcs,vmin,vmax,rows=1,columns=1,index=1,show=False,Axis="RADEC",Cube_Info='nil',image_args={},figsize=(13.33,8)):
    
    fix_axis=False
    scale=1
    COLORBARSCALE = 1
    XLABELS_NUM = 10
    vmin=vmin
    vmax=vmax
    norm='linear'
    dpi=800
    for key in image_args.keys():
        if key == "FIX_AXIS":
            fix_axis=True
        if key =="FIGSIZE":
            scale=image_args[key][0]/figsize[0]
            figsize=image_args[key]  
            
        if key == "SCALE":
            scale=image_args[key]
        if key == "COLORBARSCALE":
            COLORBARSCALE = image_args[key]
        if key =="XLABELS_NUM":
            XLABELS_NUM = image_args[key]
        if key =="VMIN":
            vmin=image_args[key]
        if key =="VMAX":
            vmax=image_args[key]
        if key =="NORM":
            norm = image_args[key]#unsured
        if key =="DPI":
            dpi = image_args[key]#unsured

            
            
    font = {'fontname':'Comic Sans MS'}
    
    plt.rcParams['font.family'] = 'DejaVu Sans Mono' 
    plt.rcParams['font.sans-serif'] = ['Arial']  

    WCS = wcs
    
    Glon = str(WCS).find("GLON")!=-1 #If the wcs is in Glon or not
    
    # Build a plot
    print(figsize)
    fig,ax = plt.subplots(nrows=rows, ncols=columns, squeeze=True, sharex='none', sharey='none', figsize=figsize)
    
    if(Cube_Info!='nil'):
        pc_per_pixel = Cube_Info["spatial_scale_y"].to(u.pc/u.pix)
        beam_position_angle = Cube_Info['Beam_Position_angle']
        
    # labels
    if Glon and Axis!="PC":

        ax.set_xlabel('Galactic Longitude',fontsize=15*scale)
        ax.set_ylabel('Galactic Latitue',fontsize=15*scale)
        
    elif(Axis=="PC"):
        
        ax.set_xlabel('Pc',fontsize=15*scale)
        ax.set_ylabel('Pc',fontsize=15*scale)
        
    else:
        
        
        ax.set_xlabel('Right Ascension',fontsize=15*scale)
        ax.set_ylabel('Declination',fontsize=15*scale)
    
    # Make axis in RA and DEC if chosen:
    
    yaxis = [int(i) for i in np.arange(0,np.shape(Cube_data)[0], int(np.shape(Cube_data)[0]/5/scale))]
    xaxis = [int(i) for i in np.arange(0,np.shape(Cube_data)[1], int(np.shape(Cube_data)[1]/XLABELS_NUM/scale))]
    
    if(Axis=="RADEC" and Glon==False):
        

        #Get the coordinates the axis pixels

        xlabels = WCS.pixel_to_world(xaxis,np.zeros(len(xaxis))).ra
        ylabels = WCS.pixel_to_world(np.zeros(len(yaxis)),yaxis).dec

        #Convert to HMS
        xlabels_hms=np.array(xlabels,dtype=type("aarf"))
        for i, e in enumerate(xlabels):
            xlabels_hms[i] = f"{int(e.hms.h)}h{int(e.hms.m)}m{np.round(e.hms.s,2)}s"
        del xlabels
        xlabels = xlabels_hms

        #Round
        ylabels_r=np.array(ylabels,dtype=type("aarf"))
        for i, e in enumerate(ylabels):
            ylabels_r[i] = (str(int(e.dms.d))+'d' +str(abs(int(e.dms.m)))+"m"+ str(abs(np.round(e.dms.s,3)))+"s")
        del ylabels
        ylabels = ylabels_r
        
    elif Glon and Axis!="PC":
        
        #Get the coordinates the axis pixels

        xlabels = WCS.pixel_to_world(xaxis,np.zeros(len(xaxis))).l.deg
        
        if(fix_axis):            
            print("changing xlables:",xlabels)
        
            #xlabels = np.concatenate([-xlabels[int(math.floor(len(xlabels)/2)):],np.flip(xlabels[int(math.ceil(len(xlabels)/2)):])]) #-180 to 180 cuz it makes more sense on a plot
            
            
            xlabels = -np.round(xlabels,1)+np.full(np.shape(xlabels),180)
            print("to:",xlabels)
        
        ylabels = WCS.pixel_to_world(np.zeros(len(yaxis)),yaxis).b.deg

        #Convert to rounded deg strings
        xlabels_string=np.array(xlabels,dtype=type("aarf"))
        for i, e in enumerate(xlabels):
            xlabels_string[i] = str(np.round(e,2))
        del xlabels
        xlabels = xlabels_string

        #Strings        
        ylabels_string=np.array(ylabels,dtype=type("aarf"))
        for i, e in enumerate(ylabels):
            ylabels_string[i] = str(np.round(e,2))
        del ylabels
        ylabels = ylabels_string
        
    elif(Axis=="PC"):
        
        xlabels = np.array([e*pc_per_pixel.value for e in xaxis],dtype=type(1))
        ylabels = np.array([e*pc_per_pixel.value for e in yaxis],dtype=type(1))
        
        if(fix_axis):  
            print("changing xlables:",xlabels)

            #xlabels = np.concatenate([np.flip(-xlabels[0:int(math.floor(len(xlabels)/2))]),(xlabels[0:int(math.ceil(len(xlabels)/2))])])  #-pc to pc cuz it makes more sense on a plot
            x_pc = [e*pc_per_pixel.value for e in xaxis]
            xlabels= np.array(np.arange(max(x_pc)/2,-max(x_pc)/2,-1000),dtype = type(1))
            xaxis = np.linspace(0,np.shape(Cube_data)[1],len(xlabels))
            print(xlabels,xaxis)
            #xlabels = np.concatenate([np.flip(-xlabels[0:int(math.floor(len(xlabels)/2))]),(xlabels[0:int(math.ceil(len(xlabels)/2))])])  #-pc to pc cuz it makes more sense on a plot
            print("to:",xlabels)
    else:
        print("error")
        del agagasgaa
        

    ax.set_xticks(xaxis)
    ax.set_yticks(yaxis)
    ax.set_xticklabels(xlabels,fontsize=12*scale);
    ax.set_yticklabels(ylabels,fontsize=12);
    ax.tick_params(direction='in',labelsize=12*scale)
    ax.tick_params(axis='both', length=8*scale, width=1.5*scale,bottom=True,left=True,right=True,top=True,colors='gray')  


    
    #ax.set_title(Title,y=(-.45),fontsize=20)        
    #ax.set_title(Title,fontsize=10,xy=(0.02,1.05),xycoords="axes fraction")        
    pylab.annotate(text=Title,fontsize=10*scale,xy=(0.02,1.05),xycoords="axes fraction")  

    
    
    color = plt.cm.inferno
    #color = plt.cm.Blues
    # show image
    im = ax.imshow(Cube_data,
                   origin        = 'lower',
                   interpolation = 'nearest',
                   cmap          = color,
                   aspect        = 'equal',
                   vmin          = vmin,
                   vmax          = vmax
                  )
    

    cbar = fig.colorbar(im,fraction=0.0096*COLORBARSCALE, pad=0.0)
    cbar.ax.tick_params(direction='in',length=8, width=1.5,colors='gray')
    cbar.set_label(Units)
    
    
    
    # plot beam
    from matplotlib.patches import Ellipse
    
    bmaj_pix = Cube_Info['desired_beam_oversampling'] # cube.header['bmaj'] # The beam oversampling is the amount of pixels in the beam
    bmin_pix = Cube_Info['desired_beam_oversampling'] # cube.header['bmin']
    bpa      = beam_position_angle.value
    
    figoff=figsize[0]/figsize[1]
    beam = Ellipse(xy     = (int(np.shape(Cube_data)[1]/25/figoff), int(np.shape(Cube_data)[0]/10/figoff)),
                   width  = bmaj_pix/2,
                   height = bmin_pix/2,
                   angle  = bpa,
                   ec     = None,
                   fc     = 'gray',
                   alpha  = 1.
                  )
    ax.add_artist(beam)


    if(show==True):
        pylab.show()
        
    # save figure
    
    #fig.savefig(("Plots/"+Title+".pdf"), dpi=1250, bbox_inches='tight')
    #fig.savefig(("Plots/"+Title+".pdf"), dpi=dpi, bbox_inches='tight')
    fig.savefig(("Plots/"+Title+".svg"), dpi=dpi, bbox_inches='tight')
    fig.savefig(("Plots/"+Title+".pdf"), dpi=dpi, bbox_inches='tight')

        
def Make_Image_Plot_Old(Pointing_Information,Annotation,Data,vmin,vmax,rows=1,columns=1,index=1,show=False):
    
    Name=Pointing_Information['target']
    WCS=Pointing_Information['wcsu']
    Glon = str(WCS).find("GLON")!=-1 #If the wcs is in Glon or not
    
    # Build a plot

    ax = pylab.subplot(rows,columns,index,projection=WCS) 
    RA = ax.coords[0]                                                                  # 
    Dec = ax.coords[1]
    im = pylab.imshow(Data,vmin=vmin,vmax=vmax,cmap='rainbow')
    RA.set_ticks(size=-3)                                                                                      
    Dec.set_ticks(size=-3) 
    RA.set_ticklabel(exclude_overlapping=True) 
    Dec.set_ticklabel(exclude_overlapping=True)                                                                                     
    
    if(Glon==False):
        pylab.xlabel('Right Ascension',fontsize=20,labelpad=1)                               
        pylab.ylabel('Declination',fontsize=20,labelpad=1)
    else:
        pylab.xlabel('Galatic longitude',fontsize=20,labelpad=1)                               
        pylab.ylabel('Galatic latitude',fontsize=20,labelpad=1)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 15)    
    cb=pylab.colorbar(im,fraction=0.1,pad=0.0)                                     
    cb.set_label(label=Name,fontsize=10,rotation=270,labelpad=20) 
    cb.ax.tick_params(which = 'major', labelsize = 10)   
    pylab.annotate(text=Annotation,fontsize=10,xy=(0.02,1.05),xycoords="axes fraction")  
    
    if(show==True):
        pylab.show()
 
def Setup_Comp_Plot(Title,axes=["",""],xlim=[0,1],ylim=[0,1],xticks= [],yticks=[],xlabels=[],ylabels=[],figsize=(8,8),args={"lims":True,"ts":10,"ls":12}):
    
    font = {'fontname':'Comic Sans MS'}
    
    plt.rcParams['font.family'] = 'DejaVu Sans Mono' 
    plt.rcParams['font.sans-serif'] = ['Arial']  

    
    # Build a plot
    
    fig,ax = plt.subplots(nrows=1, ncols=1, squeeze=True, sharex='none', sharey='none', figsize=figsize)
           
    ax.set_xlabel(axes[0],fontsize=15)
    ax.set_ylabel(axes[1],fontsize=15)
    if(args["lims"]):
        ax.set_xlim(xlim[0],xlim[1])
        ax.set_ylim(ylim[0],ylim[1])
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    ax.set_xticklabels(xlabels,fontsize=ls);
    ax.set_yticklabels(ylabels,fontsize=ls);
    ax.tick_params(direction='in',labelsize=12)
    ax.tick_params(axis='both', length=8, width=1.5,bottom=True,left=True,right=True,top=True,colors='gray')  
    ts=args["ts"]
    pylab.annotate(text=Title,fontsize=ts,xy=(0.02,1.05),xycoords="axes fraction")  


    return fig,ax
    

'''
# dataset to use:
###################################################################################################

co  = 'CO(3-2)'
gal = 'NGC253'
idx = {'idx': 4822, 'vmin': 5000, 'vmax': 12000}




# plot map and structures
###################################################################################################

# figure
fig,ax = plt.subplots(nrows=1, ncols=1, squeeze=True, sharex='none', sharey='none', figsize=(5,3))



xaxis = get_pixel_locations(mom0, axis=1).value
yaxis = get_pixel_locations(mom0, axis=2).value
xlabels = [int(i) for i in np.arange(np.max(np.round(xaxis,-1)), np.min(np.round(xaxis,-1)), -2)]
ylabels = [int(i) for i in np.arange(np.min(np.round(yaxis,-1)), np.max(np.round(yaxis,-1)), 2)]
xticks  = coordinate_to_pixel(mom0, 1, xlabels*u.pc).value
yticks  = coordinate_to_pixel(mom0, 2, ylabels*u.pc).value

ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_xticklabels(xlabels);
ax.set_yticklabels(ylabels);

# show image
im = ax.imshow(mom0.data,
               origin        = 'lower',
               interpolation = 'nearest',
               cmap          = plt.cm.Blues,
               aspect        = 'equal',
               vmin          = idx['vmin'],
               vmax          = idx['vmax']
              )
cbar = fig.colorbar(im)
cbar.set_label('intensity [K\,km\,s$^{-1}$]')

# show contour
d = dendrograms[co][gal]['dendrogram']
p = d.plotter()
p.plot_contour(ax, structure=idx['idx'], lw=1, colors='orange')

# show ellipse
idx_list = [i.idx for i in d.all_structures]
all_structs_ordered = [x for _,x in sorted(zip(idx_list,list(d.all_structures)))]
s = PPVStatistic(all_structs_ordered[idx['idx']])
ellipse = s.to_mpl_ellipse(edgecolor='red', facecolor='none')
ax.add_patch(ellipse)

# zoom in on structure
center   = ellipse.get_center()
vertices = ellipse.get_patch_transform().transform( ellipse.get_path().vertices.copy() )
bounds   = (np.min(vertices[:,0]), np.max(vertices[:,0]), np.min(vertices[:,1]), np.max(vertices[:,1]))
extent   = (bounds[1]-bounds[0], bounds[3]-bounds[2])

xmin = center[0]-2.5*extent[0]
xmax = center[0]+2.5*extent[0]
ymin = center[1]-2.5*extent[1]
ymax = center[1]+2.5*extent[1]
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# show half axes
dist = np.sqrt( (vertices[:,0]-center[0])**2 + (vertices[:,1]-center[1])**2 )
p_maj = vertices[np.argmax(dist)+1]
p_min = vertices[np.argmin(dist)]

a_maj = ax.plot((center[0],p_maj[0]), (center[1],p_maj[1]), c='red', ls='--', lw=0.5)
a_min = ax.plot((center[0],p_min[0]), (center[1],p_min[1]), c='red', ls='--', lw=0.5)

# plot beam
from matplotlib.patches import Ellipse
bmaj_pix = ( angle_to_parsec(cube.header['bmaj']*u.deg, source=gal) / cdelt ).value
bmin_pix = ( angle_to_parsec(cube.header['bmin']*u.deg, source=gal) / cdelt ).value
bpa      = cube.header['bpa']
beam = Ellipse(xy     = (xmin+0.15*(xmax-xmin), ymin+0.15*(xmax-xmin)),
               width  = bmaj_pix,
               height = bmin_pix,
               angle  = bpa,
               ec     = None,
               fc     = 'black',
               alpha  = 1.
              )
ax.add_artist(beam)

# save figure
fig.savefig(join(plotdir, 'paper', 'fig1.pdf'), dpi=300, bbox_inches='tight')


###################################################################################################
# fig A.2: structure definition comparison
###################################################################################################

catalog    = dendrograms['CO(3-2)']['NGC253']['catalog']
dendrogram = dendrograms['CO(3-2)']['NGC253']['dendrogram']

fig,axes = plt.subplots(nrows=1, ncols=2, squeeze=True, sharex='none', sharey='none', figsize=(8,4))
fig.subplots_adjust(hspace=0., wspace=0.1)

R_measures = ['size (astrodendro)',
              'size (area_ellipse)',
              'size (manual)']
lw_measures = ['linewidth (astrodendro)',
               'linewidth (mom2 mean)',
               'linewidth (mom2 median)',
               'linewidth (90% flux)',
               'linewidth (FWHM)',
               'linewidth (FW10%)']


# A1: size definition comparison
###################################################################################################

ax = axes[0]
A1colors = mpl.cm.YlOrRd(np.linspace(0.1,0.9,len(R_measures)))

min = np.nanmin([x if x!=0. else np.nan for x in flatten([catalog[R_measure].data for R_measure in R_measures])])
max = np.nanmax([catalog[R_measure].data for R_measure in R_measures])

for R_measure,color in zip(R_measures,A1colors):
    size_ad = catalog['size (astrodendro)'].data
    size    = catalog[R_measure].data
    ax.scatter(size_ad, size, marker='.', s=8, c=[color],
               label=R_measure.replace('size (','').replace(')','').replace('_',' ').replace('astrodendro',r'R$_\mathrm{astrodendro}$').replace('manual',r'R$_\mathrm{circular}$').replace('area ellipse',r'R$_\mathrm{ellipse}$'),
               zorder=3,
               rasterized=True)
ax.plot([0.5*min,2.0*max],[0.5*min,2.0*max], ls='-', color='grey', lw=1, zorder=2)

ax.set_xlabel(r'R$_\mathrm{astrodendro}$ [pc]')
ax.set_ylabel(r'R [pc]')
ax.set_xlim(0.5*min, 2.0*max)
ax.set_ylim(0.5*min, 2.0*max)
ax.legend(bbox_to_anchor=(0.,1.02,1.,0.05), loc='lower left', mode='expand', borderaxespad=0., ncol=3, scatterpoints=1, handletextpad=0., fancybox=True, fontsize=10)


# A2: linewidth definition comparison
###################################################################################################

ax = axes[1]
A2colors = mpl.cm.YlGnBu(np.linspace(0.1,1,len(lw_measures)))

min = np.nanmin([x if x>1e-2 else np.nan for x in flatten([catalog[lw_measure].data for lw_measure in lw_measures])])
max = np.nanmax([catalog[lw_measure].data for lw_measure in lw_measures])

for lw_measure,color in zip(lw_measures,A2colors):
    linewidth_ad = catalog['linewidth (astrodendro)'].data
    linewidth    = catalog[lw_measure].data
    ax.scatter(linewidth_ad, linewidth, marker='.', s=8, c=[color],
               label=lw_measure.replace('linewidth (','').replace(')','').replace('_',' ').replace('astrodendro',r'$\sigma_\mathrm{astrodendro}$').replace('mom2 mean',r'$\sigma_\mathrm{mom2\ mean}$').replace('mom2 median',r'$\sigma_\mathrm{mom2\ median}$').replace('90% flux',r'$\sigma_\mathrm{90\%}$').replace('FWHM',r'$\sigma_\mathrm{FWHM}$').replace('FW10%',r'$\sigma_\mathrm{FW10\%}$'),
               zorder=3,
               rasterized=True)
ax.plot([0.5*min,2.0*max],[0.5*min,2.0*max], ls='-', color='grey', lw=1, zorder=2)

ax.set_xlabel(r'$\sigma_\mathrm{astrodendro}$ [km\,s$^{-1}$]')
ax.set_ylabel(r'$\sigma$ [km\,s$^{-1}$]')
ax.set_xlim(0.5*min, 2.0*max)
ax.set_ylim(0.5*min, 2.0*max)
ax.yaxis.set_label_position('right')
ax.tick_params(axis='y', which='both', labelleft='off', labelright='on')
ax.legend(bbox_to_anchor=(0.,1.02,1.,0.05), loc='lower left', mode='expand', borderaxespad=0., ncol=3, scatterpoints=1, handletextpad=0., fancybox=True, fontsize=10)


# save figure
for ax in axes:
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_axisbelow(True)
    ax.grid(ls=':', c='lightgrey')
fig.savefig(join(plotdir, 'paper', 'figA.pdf'), dpi=300, bbox_inches='tight')



'''


# More useful functions

In [ ]:
# Define a function to flatten out the cubes so that they are similarly rotated at 0 degrees

def Rotate_Cube(Cube, Image_rotation):

    # R is from scipy.spatial.transform import Rotation as R
    # Flatten the cube by rotating -Image_rotation
    r = R.from_euler('z', -Image_rotation.to(u.deg), degrees=True) # Create a rotation quaternion for a rotation about the cube's z axis
    M = r.as_dcm() # the rotation in matrix form

    # Make a dictionary containing the paremeters that the header uses to rotate
    rotation_matrix = {'PC1_1':  float(M[0][0]),
                       'PC2_1':  float(M[1][0]),
                       'PC3_1':  float(M[2][0]),
                       'PC1_2':  float(M[0][1]),
                       'PC2_2':  float(M[1][1]),
                       'PC3_2':  float(M[2][1]),
                       'PC1_3':  float(M[0][2]),
                       'PC2_3':  float(M[1][2]),
                       'PC3_3':  float(M[2][2])}
    
    # Make a reprojection header
    reheader = copy.deepcopy(Cube.header)
    for key,val in rotation_matrix.items():
        reheader[key] = val

    # rotate by regridding onto rotated header
    Cube = SpectralCube.read(Cube.hdu)
    Cube.allow_huge_operations = True
    Cube3 = Cube.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
    Cube3 = SpectralCube.read(Cube3.hdu)
    return Cube3
    
        

In [ ]:



def gaussian_beam(f, beam_gauss_width):
    '''
    Fourier transform of a Gaussian beam. NOT the power spectrum (multiply exp
    argument by 2 for power spectrum).
    Parameters
    ----------
    f : np.ndarray
        Frequencies to evaluate beam at.
    beam_gauss_width : float
        Beam size. Should be the Gaussian rms, not FWHM.
    '''
    return np.exp(-f**2 * np.pi**2 * 2 * beam_gauss_width**2)

def gauss_correlated_noise_2D(shape, sigma, beam_gauss_width,
                              randomseed=327485749):
    
    '''
    Generate correlated Gaussian noise with sigma, smoothed by a
    Gaussian kernel.
    '''

    # Making a real signal. Only need real part of FFT
    freqs_yy, freqs_xx = np.meshgrid(np.fft.fftfreq(shape[0]),
                                     np.fft.rfftfreq(shape[1]), indexing="ij")

    freqs = np.sqrt(freqs_yy**2 + freqs_xx**2)
    # freqs[freqs == 0.] = np.NaN
    # freqs[freqs == 0.] = 1.

    imsize = shape[0]

    Np1 = (imsize - 1) // 2 if imsize % 2 != 0 else imsize // 2
    
    with NumpyRNGContext(randomseed):

        angles = np.random.uniform(0, 2 * np.pi,
                                   size=freqs.shape)

    noise = np.cos(angles) + 1j * np.sin(angles)

    if imsize % 2 == 0:
        noise[1:Np1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1, -1] = np.conj(noise[imsize:Np1:-1, -1])
        noise[Np1, 0] = noise[Np1, 0].real + 1j * 0.0
        noise[Np1, -1] = noise[Np1, -1].real + 1j * 0.0

    else:
        noise[1:Np1 + 1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1 + 1, -1] = np.conj(noise[imsize:Np1:-1, -1])

    # Zero freq components must have no imaginary part to be own conjugate
    noise[0, -1] = noise[0, -1].real + 1j * 0.0
    noise[0, 0] = noise[0, 0].real + 1j * 0.0

    corr_field = np.fft.irfft2(noise *
                               gaussian_beam(freqs, beam_gauss_width))

    norm = (np.sqrt(np.sum(corr_field**2)) / np.sqrt(corr_field.size)) / sigma

    corr_field /= norm
    
    return corr_field






#From Krieger 2020 (github):

####################################################################################################
# get pixel locations for axis
####################################################################################################

def get_pixel_locations(fitsimage, axis):
    """
    Get a list of pixel locations for a given image and axis.

    Parameters
    ----------
    fitsimage : string or PrimaryHDU
        File name of a fits image or astropy.fits PrimaryHDU object.
    axis : int
        Axis number. No default.

    Returns
    -------
    astropy.unit list
        List of coordinates along the axis in the units of the header (e.g. degree for a RA or DEC axis).

    """

    import astropy.units as u
    from astropy.io import fits
    import numpy as np

    if isinstance(fitsimage, str):
        header = fits.getheader(fitsimage)
    elif isinstance(fitsimage, fits.hdu.image.PrimaryHDU):
        header = fitsimage.header
    else:
        raise TypeError("Unknown format for fitsimage. Must be filename or HDUList.")

    crpix = header['crpix'+str(axis)]*u.pix
    crval = u.Quantity(str(header['crval'+str(axis)])+header['cunit'+str(axis)])
    cdelt = u.Quantity(str(header['cdelt'+str(axis)])+header['cunit'+str(axis)])/u.pix
    naxis = header['naxis'+str(axis)]

    return (np.arange(naxis)*u.pix-crpix)*cdelt+crval


####################################################################################################
#
####################################################################################################


####################################################################################################
# axis location to pixel
####################################################################################################

def coordinate_to_pixel(fitsimage, axis, coordinates, precision=2):
    """
    Calculate the pixel positions corresponding to a given coordinate along an
    axis of an fits image.

    Parameters
    ----------
    fitsimage : string or PrimaryHDU
        File name of a fits image or astropy.fits PrimaryHDU object.
    axis : int
        Axis number. No default.
    coordinates : astropy.Quantity or Quantity list
        The coordinates to be converted in the same unit as the header axis unit.
    precision : int
        Precision  of the return pixel position. Defaults to two decimal places.

    Returns
    -------
    astropy.Quantity or Quantity list
        Single value or list of pixel position corresponding to the given coordinates.

    """

    import numpy as np
    import astropy.units as u
    from astropy.io import fits

    if isinstance(fitsimage, str):
        header = fits.getheader(fitsimage)
    elif isinstance(fitsimage, fits.hdu.image.PrimaryHDU):
        header = fitsimage.header
    else:
        raise TypeError("Unknown format for fitsimage. Must be filename or HDUList.")

    crpix = header['crpix'+str(axis)]*u.pix
    crval = u.Quantity(str(header['crval'+str(axis)])+header['cunit'+str(axis)])
    cdelt = u.Quantity(str(header['cdelt'+str(axis)])+header['cunit'+str(axis)])/u.pix
    naxis = header['naxis'+str(axis)]

    if not coordinates.unit==crval.unit:
        raise TypeError("Header unit is "+str(crval.unit)+". Use matching unit.")

    return np.round((coordinates-crval)/cdelt+crpix, precision)



####################################################################################################
#
####################################################################################################

# Noise matching

this is an unused function that is from Krieger 2020, where he adds additional noise to the CMZ image to match the noise present in the NGC253
image. I do not do this because it creates a ton of false structures that dont exist and I dont want those in the CMZ data.

I require at least 5 times the noise from a noise-channel before I allow the pixels to be considered real data.
I calculate the noise for each cube after doing all the data reduction, which is expected to amplify the noise.

In [ ]:
# Return an input cube matched to the given nosie
def Noise_matching(Input_Cube,m,manual_noise=0*u.K):
    
    datn = Input_Cube.hdu.data

    npixels = np.product(Input_Cube.hdu.data.shape)
    
    
    #######
    #######
    # Calculate the RMS noise of the cube (for the line)
    #######
    #######

    Non_nan=((datn[0]>0)  | (datn[0]<0 )) # All the data that is not a nan value in the first (emissionless) channel of the cube

    m_current = np.nanstd(datn[0],where= Non_nan)*Input_Cube[0][0][0].unit #Noise (K)
    
    if manual_noise != 0*u.K:
        m_current = manual_noise
        
    actual_noise = m_current
    
    print("Current RMS noise found to be:",actual_noise,"Default to manual?: ",manual_noise,"Match to ",m)
    
    actual_noise=actual_noise.value
    target_noise = m.value
    
    additional_sigma = np.sqrt(np.abs(target_noise**2 - actual_noise**2))

    additional_noise = np.random.normal(0., additional_sigma, npixels)
    additional_noise = np.reshape(additional_noise, Input_Cube.hdu.data.shape)


    fwhm_factor = np.sqrt(8*np.log(2))
    add_noise = np.zeros(np.shape(datn))
    for lmi in range(len(datn)):
        new_seed = np.random.randint(1e9)
        additional_noise = gauss_correlated_noise_2D(shape=(Input_Cube.hdu.data[6].shape[0],Input_Cube.hdu.data[6].shape[1]), sigma=additional_sigma, beam_gauss_width=5/fwhm_factor,randomseed=new_seed)
        pp=np.where(additional_noise!=np.nan)
        add_noise[lmi][pp]=additional_noise[pp]

    new_data = datn+add_noise
    QCopy = Input_Cube.hdu
    QCopy.data = new_data
    Q = SpectralCube.read(QCopy)
    del QCopy

    

    return Q
    
    
 

# Unused

# Cluster finding

Currently unused, but this can find the highly star forming clusters as detected by Levy 2022

In [ ]:
from scipy.spatial.transform import Rotation as R
    
def Read_Clusters(FileName):
    
    sh= len(np.genfromtxt(FileName,usecols=0))
    Data=[]
    for lmi in range(50):
        try:
            Data.append(np.genfromtxt(FileName,usecols=lmi,dtype=type("2d4m")))
            #print(np.genfromtxt(FileName,usecols=lmi,dtype=type("2d4m"),skip_header=1))
        except:
            pass
    return Data
def Find_Clusters_NGC(Data):
    for lmi in range(len(Data)):
        if "ID" in Data[lmi]:
            IDs= Data[lmi][1:9999]
        if "RA" in Data[lmi]: 
            RAs= Data[lmi][1:9999]
        if "Dec" in Data[lmi]:
            Decs= Data[lmi][1:9999]
        if "r_deconv" in Data[lmi]: 
            R_deconv= Data[lmi][1:9999]#pc
        if "glon" in Data[lmi]: 
            glons= Data[lmi][1:9999]#
        if "glat" in Data[lmi]: 
            glats= Data[lmi][1:9999]#
            
    return IDs,RAs,Decs,R_deconv
#Take the cont in Jy and find the HWHM from the structures in the catalog
def Find_Clusters(Data,wcs,Cont_Data,header):
    for lmi in range(len(Data)):
        if "ID" in Data[lmi]:
            IDs= Data[lmi][1:9999]
        if "RA" in Data[lmi]: 
            RAs= Data[lmi][1:9999]
        if "Dec" in Data[lmi]:
            Decs= Data[lmi][1:9999]
        if "r_deconv" in Data[lmi]: 
            R_deconv= Data[lmi][1:9999]#pc
        if "glon" in Data[lmi]: 
            glons= Data[lmi][1:9999]#
        if "glat" in Data[lmi]: 
            glats= Data[lmi][1:9999]#
        if "herschel_column" in Data[lmi]: 
            CD= (Data[lmi][1:9999])#pc
            
        if "flux_integrated" in Data[lmi]: 
            Flux_1p3mm= Data[lmi][1:9999]#pc
    #remove nan 
    for lmii in range(len(CD)):
        try:
            if CD[lmii]=='np.nan':
                CD= np.delete(CD, lmii)
                Flux_1p3mm= np.delete(Flux_1p3mm, lmii)
                IDs= np.delete(IDs, lmii)
                glats= np.delete(glats, lmii)
                glons= np.delete(glons, lmii)
                
        except:
            CD = np.array(CD,dtype=type(1.2**5))#float
            break
    glats_New=[]
    glons_New=[]
    CDs_New=[]
    IDs_New=[]
    Flux_1p3mm_New=[]

    #print(CD,sorted(CD),type(CD),type(CD[0]))
    nth = sorted(CD)[len(CD)-34]#34 most dense leaves
    #print(nth,"A",CD,sorted(CD))
    for lmj in range(len(CD)):
        if CD[lmj]>nth:
            glats_New.append(glats[lmj])
            glons_New.append(glons[lmj])
            CDs_New.append(CD[lmj])
            IDs_New.append(int(IDs[lmj]))
            Flux_1p3mm_New.append(Flux_1p3mm[lmj])
    HWHM_rad = []      
    #print(Flux_1p3mm_New,glats_New,glons_New,CDs_New,IDs_New)
    for lmi in range(len(CDs_New)):
        glat = glats_New[lmi]
        glon = glons_New[lmi]
        Flux = float(Flux_1p3mm_New[lmi])#INtegerated flux in jy
        
        Circle_R = 0
        distance = 8.178*10**-3*u.Mpc
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(l=float(glon)*u.deg, b=float(glat)*u.deg, frame='galactic')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        
        while(True):
            Circle_R += .01
            #pixels=[(p1,p2)]
            pixels=[(p2,p1)]#Goes lat then long for the cont data
            #print(p1,p2)
            #print(np.shape(Cont_Data[p2-50:p2+50]))
            #print(np.shape(Cont_Data[50,p1-50:p1+50]))
            for lmii in range(np.shape(Cont_Data[p2-50:p2+50])[0]):
                for lmjj in range(np.shape(Cont_Data[p2-50+lmii,p1-50:p1+50])[0]):
                    #Find pixels within the circle around the center (excude the center since its there already)
                    #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                    if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                        pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
                        
            
            
            sum_flux=0
            for lmkk in range(len(pixels)):
                sum_flux += (Cont_Data[pixels[lmkk]])
            #print(p1,p2,glat,glon,np.shape(Cont_Data),pixels,Cont_Data[pixels[0]],Flux,sum_flux,Circle_R)
            if sum_flux>Flux/2:
                HWHM_rad.append(Circle_R)#Pc
                break
                
    return HWHM_rad,CDs_New,glons_New,glats_New,IDs_New

#Return masked data around clusters or one pc around clusters
def Mask_Clusters_NGC(HWHM,wcs,header,unmasked_data,ras,decs,One_Pc=False,One_Pc_Size=1,HWHM_Fac=1):
    
    Masked_Data=copy.deepcopy(unmasked_data)
    for lmi in range(len(HWHM)):
        ra = ras[lmi]
        dec = decs[lmi]
                
        Circle_R = HWHM[lmi]*HWHM_Fac
        if(One_Pc):
            
            Circle_R=One_Pc_Size
        distance = 3.5*u.Mpc
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(str(ra),str(dec), frame='icrs')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        


        #pixels=[(p1,p2)]
        pixels=[(p2,p1)]#Goes lat then long for the cont data
        #print(p1,p2)
        #print(np.shape(Cont_Data[p2-50:p2+50]))
        #print(np.shape(Cont_Data[50,p1-50:p1+50]))
        for lmii in range(np.shape(unmasked_data[0,p2-50:p2+50])[0]):
            for lmjj in range(np.shape(unmasked_data[0,p2-50+lmii,p1-50:p1+50])[0]):
                #Find pixels within the circle around the center (excude the center since its there already)
                #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                
                if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                    pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
        
        for lmi in range(len(unmasked_data)):
            
            for lmj in range(len(pixels)):
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
                Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]]=np.nan
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
     
    return Masked_Data
            



def Mask_Clusters_CMZ(HWHM,wcs,header,unmasked_data,glons,glats,One_Pc=False,One_Pc_Size=1,HWHM_Fac=1):
    
    Masked_Data=copy.deepcopy(unmasked_data)
    for lmi in range(len(HWHM)):
        glon = glons[lmi]
        glat = glats[lmi]
                
        Circle_R = HWHM[lmi]*HWHM_Fac
        if(One_Pc):
            
            Circle_R=One_Pc_Size
        distance = dist_cmz
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(float(glon)*u.deg,float(glat)*u.deg, frame='galactic')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        


        #pixels=[(p1,p2)]
        pixels=[(p2,p1)]#Goes lat then long for the cont data
        #print(p1,p2)
        #print(np.shape(Cont_Data[p2-50:p2+50]))
        #print(np.shape(Cont_Data[50,p1-50:p1+50]))
        for lmii in range(np.shape(unmasked_data[0,p2-50:p2+50])[0]):
            for lmjj in range(np.shape(unmasked_data[0,p2-50+lmii,p1-50:p1+50])[0]):
                #Find pixels within the circle around the center (excude the center since its there already)
                #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                
                if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                    pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
        
        for lmi in range(len(unmasked_data)):
            
            for lmj in range(len(pixels)):
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
                Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]]=np.nan
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
     
    return Masked_Data


In [ ]:
"""this isnt made correctly

    ######## find the original FOV
    
    fp_dec,lp_dec,fp_ra,lp_ra=0,0,0,0
    
    for lmj in range(np.shape(sc_for_cropping)[1]):
        for lmk in range(np.shape(sc_for_cropping)[2]):
                if (data[0][lmj][lmk]>0 or data[0][lmj][lmk]<0):
                    fp_dec=lmj#the first upwards pixel with data
                    break
                else:
                    pass
        if (fp_dec != 0):
            break
    for lmj in range(1,np.shape(sc_for_cropping)[1]):
        for lmk in range(np.shape(sc_for_cropping)[2]):
                if (data[0][np.shape(sc_for_cropping)[1]-lmj][lmk]>0 or data[0][np.shape(sc_for_cropping)[1]-lmj][lmk]<0):
                    lp_dec=np.shape(sc_for_cropping)[1]-lmj#the last upwards pixel with data
                    break
                else:
                    pass
        if (lp_dec != 0):
            break
    for lmk in range(np.shape(sc_for_cropping)[2]):
        for lmj in range(np.shape(sc_for_cropping)[1]):
                if (data[0][lmj][lmk]>0 or data[0][lmj][lmk]<0):
                    fp_ra=lmk # the first upwards pixel with data
                    break
                else:
                    pass
        if (fp_ra != 0):
            break
    for lmk in range(1,np.shape(sc_for_cropping)[2]):
        for lmj in range(np.shape(sc_for_cropping)[1]):
                if (data[0][lmj][np.shape(sc_for_cropping)[2]-lmk]>0 or data[0][lmj][np.shape(sc_for_cropping)[2]-lmk]<0):
                    lp_ra=np.shape(sc_for_cropping)[2]-lmk # the last upwards pixel with data
                    break
                else:
                    pass
        if (lp_ra != 0):
            break
    print(fp_dec,lp_dec,fp_ra,lp_ra)
            
            
    if(r!=0*u.deg):
        if(lp_ra-fp_ra>lp_dec-fp_dec):
            disk_pixels=(lp_dec-fp_dec)/np.sin(r_rad)
            jet_pixels=(lp_dec-fp_dec)/np.sin(np.pi/2 - r_rad)
        else:
            disk_pixels=(lp_ra-fp_ra)/np.sin(r_rad)
            jet_pixels=(lp_ra-fp_ra)/np.sin(np.pi/2 - r_rad)
    else:
        disk_pixels=lp_ra-fp_ra
        jet_pixels=lp_dec-fp_dec
        
    print(disk_pixels,jet_pixels)
    orig_fov=[0,0]            
    orig_fov[0] = np.round(((disk_pixels*(cdelt_x.to(u.rad)*d))/u.rad).to(u.pc),1) #the fov across the disk
    orig_fov[1] = np.round(((jet_pixels*(cdelt_x.to(u.rad)*d))/u.rad).to(u.pc),1) #the fov coming up from the disk
    
    
    print(orig_fov)
    
    print("cropped cube from",orig_fov,"to:",desired_fov)
    
"""

# Spider crawl, a function used for modifying the input parameters to the dendrogram

In [13]:
import re

def spider_crawl(file_paths, search_text, replacement_text,modify=False):
    """
    Goes into a series of files and replaces only the matching part of a line.
    
    Parameters:
    file_paths (list): List of file paths to modify.
    search_text (str): The text to search for.
    replacement_text (str): The text to replace the found text with.
    """
    for file_path in file_paths:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
            
            modified = False
            if modify:
                with open(file_path, 'w', encoding='utf-8') as file:
                    for line in lines:
                        if search_text in line:
                            new_line = re.sub(re.escape(search_text), replacement_text, line)
                            if(modify):
                                file.write(new_line)
                                print("Modified",line, "\n to",new_line)
                                modified = True

                            else:
                                file.write(line)
                                print(new_line)
                        else:
                            file.write(line)
            else:
                with open(file_path, 'r', encoding='utf-8') as file:
                    for line in lines:
                        if search_text in line:
                            print(line)

                            

            if modified:
                print(f"Modified: {file_path}")
            else:
                print(f"No changes made: {file_path}")

        except Exception as e:
            print(f"Error modifying {file_path}: {e}")

# Example usage:
# spider_crawl(["file1.py", "file2.txt"], "old_text", "new_text")


def spider_crawl_find(file_paths, search_text,modify=False):
    """
    Goes into a series of files and replaces only the matching part of a line.
    
    Parameters:
    file_paths (list): List of file paths to modify.
    search_text (str): The text to search for.
    replacement_text (str): The text to replace the found text with.
    """
    for file_path in file_paths:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
  
                with open(file_path, 'r', encoding='utf-8') as file:
                    for line in lines:
                        if search_text in line:
                            print(line)

                            

        except Exception as e:
            print(f"Error finding {file_path}: {e}")

# Example usage:
# spider_crawl(["file1.py", "file2.txt"], "old_text", "new_text")


In [25]:
#Dendrogram_Calculation(Cube_Information,Minimum_Pixel_Requirement=1,min_delta=5,noise_requirement=5)
#ol = "Ooooooooooooooh"
#ol2 = "Dendrogram_Calculation(Cube_Information,Minimum_Pixel_Requirement=.5,min_delta=5,noise_requirement=5)"
#nl = "Dendrogram_Calculation(Cube_Information,Minimum_Pixel_Requirement=0,min_delta=5,noise_requirement=5)"
#ols = ol# [ol,ol2]
#spider_crawl(["test.txt"], ols, nl)

Modified: test.txt


Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/home/ben/.local/lib/python3.8/site-packages/IPython/core/history.py", line 780, in writeout_cache
    self._writeout_input_cache(conn)
  File "/home/ben/.local/lib/python3.8/site-packages/IPython/core/history.py", line 763, in _writeout_input_cache
    conn.execute("INSERT INTO history VALUES (?, ?, ?, ?)",
sqlite3.OperationalError: attempt to write a readonly database


In [ ]:
'''import os

def spider_crawl(file_paths, search_text, replacement_text):
    """
    #Searches for a specific line of text in multiple files and replaces it with a new line.
    #Creates a backup of each file before modifying it.
    
    #:param file_paths: List of file paths to process
    #:param search_text: The text to search for in each file
    #:param replacement_text: The text to replace the found line with
    """
    for file_path in file_paths:
        if not os.path.isfile(file_path):
            print(f"Skipping {file_path}: Not a valid file.")
            continue
        
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
            
            modified = False
            new_lines = []
            for line in lines:
                for s in search_text:
                    if s in line:
                        new_lines.append(replacement_text + '\n')  # Replace the line
                        modified = True
                        break
                else:
                    new_lines.append(line)
            
            if modified:
                backup_path = file_path + ".bak"
                os.rename(file_path, backup_path)  # Create a backup
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.writelines(new_lines)
                print(f"Updated {file_path}. Backup saved as {backup_path}.")
            else:
                print(f"No changes made to {file_path}: Search text not found.")
        
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

# Example usage:
# spider_crawl(["file1.txt", "file2.py"], "old_line_of_code", "new_line_of_code")
'''